## Field Lab: Quantitative Investment Startegy 
#### Maria Julia Luna Gonzalez (42929): Sector-Based Credit Risk and Stockss Returns 
#### Tiago Senra Casanova Vasco (39569): Analyzing the Dynamic Relationship Between Consumer Spending Patterns and Stock Market Sector Performance

## Sector-Based Credit Risk and Stocks Returns 

In [ ]:
import wrds
import pandas as pd
import plotly.express as px
import statsmodels.api as sm
import numpy as np
import plotly.graph_objects as go

# Connection to WRDS

db = wrds.Connection(wrds_username='')

In [ ]:
# Defining the start_date and end_date

start_date = '1987-05-01'
end_date = '2017-02-28'

# Getting the Credit Risks and the Sectors

query = f'''
    SELECT 
    datadate,
    gvkey,
    splticrm
    FROM comp_na_daily_all.adsprate
    WHERE datadate BETWEEN '{start_date}' AND '{end_date}'
    AND splticrm IS NOT NULL
'''

data = db.raw_sql(query)
df_GSIC = db.get_table(library="comp", table="company", columns =['gvkey','gsector'])
data = data.merge(df_GSIC, on='gvkey', how='left')
data = data.dropna()

In [ ]:
# Getting the Monthly Returns
query2 = f'''
    SELECT 
    datadate,
    gvkey,
    trt1m,
    fic,
    curcdm
    FROM comp_na_daily_all.secm
    WHERE datadate BETWEEN '{start_date}' AND '{end_date}'
    AND fic = 'USA'
    AND curcdm = 'USD'
'''

data2 = db.raw_sql(query2)

#Joining the DataFrames 
merged_data = pd.merge(data, data2, on=['datadate', 'gvkey'], how='left')
merged_data = merged_data.dropna()
merged_data['trt1m'] = merged_data['trt1m'] / 100

In [ ]:
# Getting the FF3 variables 
query3 = f'''
    SELECT 
    dateff,
    smb,
    hml,
    mktrf,
    rf
    FROM ff_all.factors_monthly
    WHERE dateff BETWEEN '{start_date}' AND '{end_date}'
'''

data3 = db.raw_sql(query3)
data3['dateff'] = pd.to_datetime(data3['dateff'])
data3['dateff'] = data3['dateff'] + pd.offsets.MonthEnd(0)
data3 = data3.rename(columns={'dateff': 'datadate'})

### Organizing & Undestanding the data 

In [ ]:
# Cleaning the data: Keeping only the rows that don't contain 'N.M.' or 'SD'

mask = (merged_data['splticrm'] != 'N.M.') & (merged_data['splticrm'] != 'SD') & (merged_data['splticrm'] != 'Suspended')
merged_data = merged_data[mask]

# Replacing Credit Risk with numeric values

splticrm_mapping = {
    "AAA": 1.0,
    "AA+": 2.0,
    "AA": 3.0,
    "AA-": 4.0,
    "A+": 5.0,
    "A": 6.0,
    "A-": 7.0,
    "BBB+": 8.0,
    "BBB": 9.0,
    "BBB-": 10.0,
    "BB+": 11.0,
    "BB": 12.0,
    "BB-": 13.0,
    "B+": 14.0,
    "B": 15.0,
    "B-": 16.0,
    "CCC+": 17.0,
    "CCC": 18.0,
    "CCC-": 19.0,
    "CC": 20.0,
    "C": 21.0,
    "D": 22.0
}

merged_data.loc[:, 'splticrm'] = merged_data['splticrm'].replace(splticrm_mapping)

In [ ]:
# Matching sector codes to sector names

sector_names = {
    10: '10 Energy',
    15: '15 Materials',
    20: '20 Industrials',
    25: '25 Consumer Discretionary',
    30: '30 Consumer Staples',
    35: '35 Health Care',
    40: '40 Financials',
    45: '45 Information Technology',
    50: '50 Communication Services',
    55: '55 Utilities',
    60: '60 Real Estate'
}

In [ ]:
monthly_returns = merged_data.groupby(['datadate', 'gsector'])['trt1m'].mean().reset_index()
monthly_returns = monthly_returns.rename(columns={'trt1m': 'monthly_return'})
pivot_table10 = monthly_returns.pivot_table(index='datadate', columns='gsector', values='monthly_return')
pivot_table10.reset_index(inplace=True)
pivot_table10['datadate'] = pd.to_datetime(pivot_table10['datadate'])
Data40 = pd.merge(pivot_table10, data3[['datadate', 'mktrf']], on='datadate', how='left')

# Getting the betas of each sector 
sector_betas = []

for sector_column in Data40.columns[1:-1]: 
    sector_number = sector_column  
    y = Data40[sector_column]  
    X = sm.add_constant(Data40['mktrf'])  
    model = sm.OLS(y, X).fit()  
    beta = model.params['mktrf']  
    sector_betas.append({'gsector': sector_number, 'Beta': beta})

    beta_df = pd.DataFrame(sector_betas)
beta_df

### Portfolio Formation

In [ ]:
# Getting the Excess Returns
merged_data['datadate'] = pd.to_datetime(merged_data['datadate'], format='%Y-%m-%d')
merged_data = merged_data.sort_values(by='datadate')

excess_returns = []

for index, row in merged_data.iterrows():
    date = row['datadate']
    
    matching_row = data3[data3['datadate'] == date]
    
    if not matching_row.empty:
        excess_return = row['trt1m'] - matching_row.iloc[0]['rf']
    else:
        excess_return =  row['trt1m'] 
        
    excess_returns.append(excess_return)

merged_data['excess_returns'] = excess_returns

In [ ]:
# Forming the Equal Weights Long-Short Portfolio for each Sector

merged_data['signal'] = np.where(merged_data['splticrm'] > 10, -1, 1)
merged_data['portfolio_returns'] = merged_data['signal'] * merged_data['excess_returns']
long_short_portfolio = merged_data.groupby(['datadate', 'gsector'])['portfolio_returns'].mean().reset_index()
long_short_portfolio.set_index('datadate', inplace=True)
long_short_portfolio['adjusted_portfolio_returns'] = long_short_portfolio['portfolio_returns'] + 1
long_short_portfolio['cumulative_returns'] = long_short_portfolio.groupby('gsector')['adjusted_portfolio_returns'].cumprod() - 1
long_short_portfolio.reset_index(inplace=True)
data3['cumulative_mktrf'] = (1 + data3['mktrf']).cumprod() - 1


# Plotting
fig = px.line(long_short_portfolio, x='datadate', y='cumulative_returns', color='gsector',
              title='Graph 1: Sectors Portfolios Cumulative Return',
              labels={'datadate': 'Date', 'cumulative_returns': 'Cumulative Returns'},
              line_shape='linear')

fig.add_scatter(x=data3['datadate'],
                y=data3['cumulative_mktrf'],  
                line=dict(dash='dot'),  
                name='mktrf')
                
fig.update_layout(
    xaxis_title='Date',  
    yaxis_title='Portfolio Cumulative Return',
    xaxis=dict(showline=True, showgrid= True, gridwidth=0.2, gridcolor='lightgray', showticklabels=True, linecolor='black'),
    yaxis=dict(showline=True, showgrid= True, gridwidth=0.2, gridcolor='lightgray', showticklabels=True, linecolor='black'),
    plot_bgcolor='white',  
    paper_bgcolor='white', 
    legend_title='',
    legend=dict(
        bgcolor='rgba(0,0,0,0)', 
       
    )
)

fig.add_shape(
    type="rect",
    x0=0,
    x1=1,
    y0=0,
    y1=1,
    xref="paper",
    yref="paper",
    line=dict(color="black", width=1.5),
    layer="below"
)


for code, name in sector_names.items():
    for trace in fig.data:
        if trace.name == str(code):
            trace.name = name

fig.update_layout(width=1000, height=500)

fig.show()

In [ ]:
# Performance Measures - Annual return, Volatility and Sharpe Ratio - Full Period

def calculate_annual_return(returns):
    geometric_mean_return = np.prod(1 + returns) ** (1 / len(returns)) - 1
    annual_return = (1 + geometric_mean_return) ** 12 - 1
    return annual_return


annualized_returns = long_short_portfolio.groupby('gsector')['portfolio_returns'].apply(calculate_annual_return)
performance_metrics = pd.DataFrame({'gsector': annualized_returns.index, 'Annualized Return': annualized_returns.values})

def calculate_annualized_volatility(returns):
    monthly_volatility = np.std(returns)
    annual_volatility = monthly_volatility * np.sqrt(12) 
    return annual_volatility

annualized_volatility = long_short_portfolio.groupby('gsector')['portfolio_returns'].apply(calculate_annualized_volatility)
volatility_df = annualized_volatility.reset_index()  

volatility_df.columns = ['gsector', 'Volatility']

performance_metrics = performance_metrics.merge(volatility_df, on='gsector', how='left')

performance_metrics['Sharpe Ratio'] = performance_metrics['Annualized Return'] / performance_metrics['Volatility']

mktrf_annual_return = calculate_annual_return(data3['mktrf'])

mktrf_annualized_volatility = calculate_annualized_volatility(data3['mktrf'])

mktrf_sharpe_ratio = mktrf_annual_return / mktrf_annualized_volatility

mktrf_stats = pd.DataFrame({'gsector': ['mktrf'], 'Annualized Return': [mktrf_annual_return], 'Volatility': [mktrf_annualized_volatility], 'Sharpe Ratio': [mktrf_sharpe_ratio]})

performance_metrics = pd.concat([performance_metrics, mktrf_stats], ignore_index=True)

performance_metrics.set_index('gsector', inplace=True)

performance_metrics

##### CREATING THE PORTFOLIOS WITH THE BEST SECTORS

In [ ]:
# Selecting the Best Performing Portfolios per month

long_short_portfolio_new = long_short_portfolio.copy()
start = pd.to_datetime('1988-05-01')

long_short_portfolio_new['sharpe_ratio'] = long_short_portfolio_new.groupby('gsector')['portfolio_returns'].transform(
    lambda x: x.expanding().mean() / x.expanding().std())
long_short_portfolio_new['cumulative_returns'] = long_short_portfolio_new.groupby('gsector')['adjusted_portfolio_returns'].transform(
    lambda x: x.expanding().apply(np.prod) - 1)

long_short_portfolio_new = long_short_portfolio_new[long_short_portfolio_new['datadate'] >= start]
long_short_portfolio_new['rank_sharpe'] = long_short_portfolio_new.groupby('datadate')['sharpe_ratio'].rank(ascending=False)
long_short_portfolio_new['rank_cum'] = long_short_portfolio_new.groupby('datadate')['cumulative_returns'].rank(ascending=False)

long_short_portfolio_new['final_rank'] = long_short_portfolio_new[['rank_sharpe', 'rank_cum']].mean(axis=1)

top3_sectors = long_short_portfolio_new.groupby('datadate').apply(lambda x: x.nsmallest(3, 'final_rank')).reset_index(drop=True)
top3_sectors = top3_sectors.dropna(subset=['final_rank'])

unique_combinations = top3_sectors[['datadate', 'gsector']]

merged_data['datadate'] = pd.to_datetime(merged_data['datadate'])
unique_combinations['datadate'] = pd.to_datetime(unique_combinations['datadate'])

filtered_data_new= merged_data[merged_data.set_index(['datadate', 'gsector']).index.isin(unique_combinations.set_index(['datadate', 'gsector']).index)]



# Long-short portfolio

filtered_data_new1 = filtered_data_new.copy()
filtered_data_new1['signal'] = np.where(filtered_data_new1['splticrm'] > 10, -1, 1)
filtered_data_new1['portfolio_returns'] = filtered_data_new1['signal'] * filtered_data_new1['excess_returns']
new_long_short = filtered_data_new1.groupby(['datadate'])['portfolio_returns'].mean().reset_index()
new_long_short.set_index('datadate', inplace=True)
new_long_short['adjusted_portfolio_returns'] = new_long_short['portfolio_returns'] + 1
new_long_short['cumulative_returns'] = new_long_short['adjusted_portfolio_returns'].cumprod() - 1
new_long_short.reset_index(inplace=True)


# Long portfolio
filtered_data_new2 = filtered_data_new.copy()
filtered_data_new2 = filtered_data_new2[filtered_data_new2['splticrm'] < 11]
long_portfolio = filtered_data_new2[['datadate', 'excess_returns']]
long_portfolio = long_portfolio.groupby(['datadate'])['excess_returns'].mean().reset_index()
long_portfolio['adjusted_portfolio_returns'] = long_portfolio['excess_returns'] + 1
long_portfolio['cumulative_returns'] = long_portfolio['adjusted_portfolio_returns'].cumprod() - 1

# Market 
data3 = data3[data3['datadate'] >= '1988-05-31']
data3['cumulative_mktrf'] = (1 + data3['mktrf']).cumprod() - 1


long_portfolio['datadate'] = pd.to_datetime(long_portfolio['datadate'])
new_long_short['datadate'] = pd.to_datetime(new_long_short['datadate'])
data3['datadate'] = pd.to_datetime(data3['datadate'])

long_portfolio['datadate'] = long_portfolio['datadate'].dt.strftime('%Y-%m-%d')
new_long_short['datadate'] = new_long_short['datadate'].dt.strftime('%Y-%m-%d')
data3['datadate'] = data3['datadate'].dt.strftime('%Y-%m-%d')


# Plotting
trace_long_portfolio = go.Scatter(x=long_portfolio['datadate'], y=long_portfolio['cumulative_returns'],
                                  name='Long Portfolio', line_shape='linear')

trace_long_short_portfolio = go.Scatter(x=new_long_short['datadate'], y=new_long_short['cumulative_returns'],
                                        name='Long-Short Portfolio', line_shape='linear')

trace_mktrf = go.Scatter(x=data3['datadate'], y=data3['cumulative_mktrf'],
                        name='Market Returns', line_shape='linear', line=dict(dash='dot'))

layout = go.Layout(
    title='Graph 2: Portfolios Cumulative Returns',
    xaxis=dict(title='Date', showline=True, showgrid=True, gridwidth=0.2, gridcolor='lightgray', showticklabels=True, linecolor='black'),
    yaxis=dict(
        title='Portfolio Cumulative Return',
        showline=True,
        showgrid=True,
        gridwidth=0.2,
        gridcolor='lightgray',
        showticklabels=True,
        linecolor='black',
        zeroline=True,  
        zerolinecolor='lightgray',  
        zerolinewidth=1  
    ),
    xaxis_linecolor='black',
    yaxis_linecolor='black',
    plot_bgcolor='white',
    paper_bgcolor='white',
    legend_title='',
    legend=dict(
        orientation="h",
        x=0.5,
        y=-0.15,
        xanchor='center',
        yanchor='top',
        bgcolor='rgba(0,0,0,0)'
    ),
)

fig = go.Figure(data=[trace_long_portfolio, trace_long_short_portfolio, trace_mktrf], layout=layout)

fig.add_shape(
    type="line",
    x0=long_portfolio['datadate'].min(),
    x1=long_portfolio['datadate'].min(),
    y0=0,
    y1=1,
    xref="x",
    yref="paper",
    line=dict(color="black", width=1.5),
    layer="below"
)

fig.add_shape(
    type="rect",
    x0=0,
    x1=1,
    y0=0,
    y1=1,
    xref="paper",
    yref="paper",
    line=dict(color="black", width=1.5),
    layer="below"
)

fig.update_layout(width=800, height=500)

fig.show()

In [ ]:
# Performance Metrics - Full Period

data = {
    'Portfolio': ['Long Portfolio', 'Long-Short Portfolio', 'Market Returns'],
    'Annual Return': [
        calculate_annual_return(long_portfolio['excess_returns']),
        calculate_annual_return(new_long_short['portfolio_returns']),
        calculate_annual_return(data3['mktrf'])
    ],
    'Volatility': [
        calculate_annualized_volatility(long_portfolio['excess_returns']),
        calculate_annualized_volatility(new_long_short['portfolio_returns']),
        calculate_annualized_volatility(data3['mktrf'])
    ]
}

data['Sharpe Ratio'] = [
    data['Annual Return'][i] / data['Volatility'][i] for i in range(len(data['Annual Return']))
]

performance = pd.DataFrame(data)
performance

In [ ]:
# Dividing the Data on First and Second Halves - Performance Measuments

# Long-short

new_long_short.sort_values(by='datadate', inplace=True)

midpoint = len(new_long_short) // 2

longshort_first_half = new_long_short.iloc[:midpoint]
longshort_second_half = new_long_short.iloc[midpoint:]

# Long

long_portfolio.sort_values(by='datadate', inplace=True)

midpoint = len(long_portfolio) // 2

long_first_half = long_portfolio.iloc[:midpoint]
long_second_half = long_portfolio.iloc[midpoint:]

# Factors

data3.sort_values(by='datadate', inplace=True)

midpoint_data3 = len(data3) // 2

first_half_data3 = data3.iloc[:midpoint_data3]
second_half_data3 = data3.iloc[midpoint_data3:]

In [ ]:
# Performance Metrics - First Half

data_first_half = {
    'Portfolio': ['Long Portfolio', 'Long-Short Portfolio', 'Market Returns'],
    'Annual Return first_half': [
        calculate_annual_return(long_first_half['excess_returns']),
        calculate_annual_return(longshort_first_half['portfolio_returns']),
        calculate_annual_return(first_half_data3['mktrf'])
    ],
    'Volatility first_half': [
        calculate_annualized_volatility(long_first_half['excess_returns']),
        calculate_annualized_volatility(longshort_first_half['portfolio_returns']),
        calculate_annualized_volatility(first_half_data3['mktrf'])
    ]
}

data_first_half['Sharpe Ratio first_half'] = [
    data_first_half['Annual Return first_half'][i] / data_first_half['Volatility first_half'][i] for i in range(len(data_first_half['Annual Return first_half']))
]

performance_first_half = pd.DataFrame(data_first_half)
performance_first_half

In [ ]:
# Performance Metrics - Second Half

data_second_half = {
    'Portfolio': ['Long Portfolio', 'Long-Short Portfolio', 'Market Returns'],
    'Annual Return second_half': [
        calculate_annual_return(long_second_half['excess_returns']),
        calculate_annual_return(longshort_second_half['portfolio_returns']),
        calculate_annual_return(second_half_data3['mktrf'])
    ],
    'Volatility second_half': [
        calculate_annualized_volatility(long_second_half['excess_returns']),
        calculate_annualized_volatility(longshort_second_half['portfolio_returns']),
        calculate_annualized_volatility(second_half_data3['mktrf'])
    ]
}

data_second_half['Sharpe Ratio second_half'] = [
    data_second_half['Annual Return second_half'][i] / data_second_half['Volatility second_half'][i] for i in range(len(data_second_half['Annual Return second_half']))
]

performance_second_half = pd.DataFrame(data_second_half)
performance_second_half

In [ ]:
# Organizing the data for CAPM

new_long_short = new_long_short.merge(data3[['datadate', 'mktrf']], on='datadate', how='left')
long_portfolio = long_portfolio.merge(data3[['datadate', 'mktrf']], on='datadate', how='left')

In [ ]:
# CAPM - Full period

results_CAPM = []

# Long Portfolio
y_long = long_portfolio['excess_returns']
X_long = long_portfolio[['mktrf']]
X_long = sm.add_constant(X_long)

model_long = sm.OLS(y_long, X_long).fit()

alpha_long = model_long.params['const']
beta_long = model_long.params['mktrf']
t_statistic_alpha_long = model_long.tvalues['const']
t_statistic_beta_long = model_long.tvalues['mktrf']

excess_returns_long = y_long
tracking_error_long = excess_returns_long.std()
ir_long = alpha_long / tracking_error_long

results_CAPM.append({
    'Portfolio': 'Long Portfolio',
    'Alpha (CAPM)': alpha_long,
    'T-Statistic (Alpha)': t_statistic_alpha_long,
    'Beta (CAPM)': beta_long,
    'T-Statistic (Beta)': t_statistic_beta_long,
    'Information Ratio (CAPM)': ir_long
})

# Long-Short Portfolio
y_longshort = new_long_short['portfolio_returns']
X_longshort = new_long_short[['mktrf']]
X_longshort = sm.add_constant(X_longshort)

model_longshort = sm.OLS(y_longshort, X_longshort).fit()

alpha_longshort = model_longshort.params['const']
beta_longshort = model_longshort.params['mktrf']
t_statistic_alpha_longshort = model_longshort.tvalues['const']
t_statistic_beta_longshort = model_longshort.tvalues['mktrf']

excess_returns_longshort = y_longshort
tracking_error_longshort = excess_returns_longshort.std()
ir_longshort = alpha_longshort / tracking_error_longshort

results_CAPM.append({
    'Portfolio': 'Long-short Portfolio',
    'Alpha (CAPM)': alpha_longshort,
    'T-Statistic (Alpha)': t_statistic_alpha_longshort,
    'Beta (CAPM)': beta_longshort,
    'T-Statistic (Beta)': t_statistic_beta_longshort,
    'Information Ratio (CAPM)': ir_longshort
})

results_CAPM_df = pd.DataFrame(results_CAPM)
results_CAPM_df

In [ ]:
# Splitting Long-short for CAPM

new_long_short.sort_values(by='datadate', inplace=True)

midpoint = len(new_long_short) // 2

longshort_first_half = new_long_short.iloc[:midpoint]
longshort_second_half = new_long_short.iloc[midpoint:]

# Splitting Long for CAPM

long_portfolio.sort_values(by='datadate', inplace=True)

midpoint = len(long_portfolio) // 2

long_first_half = long_portfolio.iloc[:midpoint]
long_second_half = long_portfolio.iloc[midpoint:]

In [ ]:
# CAPM in-sample & out-of-sample 

results_CAPM = []

# Long Portfolio
for period, half_data in enumerate([long_first_half, long_second_half]):
    y_long = half_data['excess_returns']
    X_long = half_data[['mktrf']]
    X_long = sm.add_constant(X_long)

    model_long = sm.OLS(y_long, X_long).fit()

    alpha_long = model_long.params['const']
    beta_long = model_long.params['mktrf']
    t_statistic_alpha_long = model_long.tvalues['const']
    t_statistic_beta_long = model_long.tvalues['mktrf']

    excess_returns_long = y_long
    tracking_error_long = excess_returns_long.std()
    ir_long = alpha_long / tracking_error_long

    results_CAPM.append({
        'Portfolio': 'Long Portfolio',
        'Period': 'first half' if period == 0 else 'second half',
        'Alpha (CAPM)': alpha_long,
        'T-Statistic (Alpha)': t_statistic_alpha_long,
        'Beta (CAPM)': beta_long,
        'T-Statistic (Beta)': t_statistic_beta_long,
        'Information Ratio (CAPM)': ir_long,
    })

# Long-Short Portfolio
for period, half_data in enumerate([longshort_first_half, longshort_second_half]):
    y_longshort = half_data['portfolio_returns']
    X_longshort = half_data[['mktrf']]
    X_longshort = sm.add_constant(X_longshort)

    model_longshort = sm.OLS(y_longshort, X_longshort).fit()

    alpha_longshort = model_longshort.params['const']
    beta_longshort = model_longshort.params['mktrf']
    t_statistic_alpha_longshort = model_longshort.tvalues['const']
    t_statistic_beta_longshort = model_longshort.tvalues['mktrf']

    excess_returns_longshort = y_longshort
    tracking_error_longshort = excess_returns_longshort.std()
    ir_longshort = alpha_longshort / tracking_error_longshort

    results_CAPM.append({
        'Portfolio': 'Long-short Portfolio',
        'Period': 'first half' if period == 0 else 'second half',
        'Alpha (CAPM)': alpha_longshort,
        'T-Statistic (Alpha)': t_statistic_alpha_longshort,
        'Beta (CAPM)': beta_longshort,
        'T-Statistic (Beta)': t_statistic_beta_longshort,
        'Information Ratio (CAPM)': ir_longshort,
    })

results_CAPM_df = pd.DataFrame(results_CAPM)
results_CAPM_df



In [ ]:
# Organizing the data for the FF3

new_long_short = new_long_short.merge(data3[['datadate', 'smb', 'hml']], on='datadate', how='left')
long_portfolio = long_portfolio.merge(data3[['datadate', 'smb', 'hml']], on='datadate', how='left')

In [ ]:
# FF3 - Full period

results_ff3 = []

# Long Portfolio
y_long = long_portfolio['excess_returns']
X_long = long_portfolio[['mktrf', 'smb', 'hml']]
X_long = sm.add_constant(X_long)

model_long = sm.OLS(y_long, X_long).fit()

alpha_long = model_long.params['const']
t_statistic_alpha_long = model_long.tvalues['const']

beta_mkt_long = model_long.params['mktrf']
t_statistic_mkt_long = model_long.tvalues['mktrf']

beta_smb_long = model_long.params['smb']
t_statistic_smb_long = model_long.tvalues['smb']

beta_hml_long = model_long.params['hml']
t_statistic_hml_long = model_long.tvalues['hml']

excess_returns_long = y_long
tracking_error_long = excess_returns_long.std()
ir_long = alpha_long / tracking_error_long

results_ff3.append({
    'Portfolio': 'Long Portfolio',
    'Alpha (FF3)': alpha_long,
    'T-Statistic Alpha': t_statistic_alpha_long,
    'Beta Mkt-Rf': beta_mkt_long,
    'T-Statistic Mkt-Rf': t_statistic_mkt_long,
    'Beta SMB': beta_smb_long,
    'T-Statistic SMB': t_statistic_smb_long,
    'Beta HML': beta_hml_long,
    'T-Statistic HML': t_statistic_hml_long,
    'Information Ratio (FF3)': ir_long
})

# Long-Short Portfolio
y_longshort = new_long_short['portfolio_returns']
X_longshort = new_long_short[['mktrf', 'smb', 'hml']]
X_longshort = sm.add_constant(X_longshort)

model_longshort = sm.OLS(y_longshort, X_longshort).fit()

alpha_longshort = model_longshort.params['const']
t_statistic_alpha_longshort = model_longshort.tvalues['const']

beta_mkt_longshort = model_longshort.params['mktrf']
t_statistic_mkt_longshort = model_longshort.tvalues['mktrf']

beta_smb_longshort = model_longshort.params['smb']
t_statistic_smb_longshort = model_longshort.tvalues['smb']

beta_hml_longshort = model_longshort.params['hml']
t_statistic_hml_longshort = model_longshort.tvalues['hml']

excess_returns_longshort = y_longshort
tracking_error_longshort = excess_returns_longshort.std()
ir_longshort = alpha_longshort / tracking_error_longshort

results_ff3.append({
    'Portfolio': 'Long-Short Portfolio',
    'Alpha (FF3)': alpha_longshort,
    'T-Statistic Alpha': t_statistic_alpha_longshort,
    'Beta Mkt-Rf': beta_mkt_longshort,
    'T-Statistic Mkt-Rf': t_statistic_mkt_longshort,
    'Beta SMB': beta_smb_longshort,
    'T-Statistic SMB': t_statistic_smb_longshort,
    'Beta HML': beta_hml_longshort,
    'T-Statistic HML': t_statistic_hml_longshort,
    'Information Ratio (FF3)': ir_longshort
})

results_ff3_df = pd.DataFrame(results_ff3)
results_ff3_df

In [ ]:
# Splitting Long-short for FF3

new_long_short.sort_values(by='datadate', inplace=True)

midpoint = len(new_long_short) // 2

longshort_first_half = new_long_short.iloc[:midpoint]
longshort_second_half = new_long_short.iloc[midpoint:]

# Splitting Long for FF3

long_portfolio.sort_values(by='datadate', inplace=True)

midpoint = len(long_portfolio) // 2

long_first_half = long_portfolio.iloc[:midpoint]
long_second_half = long_portfolio.iloc[midpoint:]

In [ ]:
# FF3 in-sample & out-of-sample 

results_ff3 = []

# Long Portfolio
for period, half_data in enumerate([long_first_half, long_second_half]):
    y_long = half_data['excess_returns']
    X_long = half_data[['mktrf', 'smb', 'hml']]
    X_long = sm.add_constant(X_long)

    model_long = sm.OLS(y_long, X_long).fit()

    alpha_long = model_long.params['const']
    t_statistic_alpha_long = model_long.tvalues['const']

    beta_mkt_long = model_long.params['mktrf']
    t_statistic_mkt_long = model_long.tvalues['mktrf']

    beta_smb_long = model_long.params['smb']
    t_statistic_smb_long = model_long.tvalues['smb']

    beta_hml_long = model_long.params['hml']
    t_statistic_hml_long = model_long.tvalues['hml']

    excess_returns_long = y_long
    tracking_error_long = excess_returns_long.std()
    ir_long = alpha_long / tracking_error_long

    results_ff3.append({
        'Portfolio': 'Long Portfolio',
        'Period': 'first half' if period == 0 else 'second half',
        'Alpha (FF3)': alpha_long,
        'T-Statistic Alpha': t_statistic_alpha_long,
        'Beta Mkt-Rf': beta_mkt_long,
        'T-Statistic Mkt-Rf': t_statistic_mkt_long,
        'Beta SMB': beta_smb_long,
        'T-Statistic SMB': t_statistic_smb_long,
        'Beta HML': beta_hml_long,
        'T-Statistic HML': t_statistic_hml_long,
        'Information Ratio (FF3)': ir_long
    })

# Long-Short Portfolio
for period, half_data in enumerate([longshort_first_half, longshort_second_half]):
    y_longshort = half_data['portfolio_returns']
    X_longshort = half_data[['mktrf', 'smb', 'hml']]
    X_longshort = sm.add_constant(X_longshort)

    model_longshort = sm.OLS(y_longshort, X_longshort).fit()

    alpha_longshort = model_longshort.params['const']
    t_statistic_alpha_longshort = model_longshort.tvalues['const']

    beta_mkt_longshort = model_longshort.params['mktrf']
    t_statistic_mkt_longshort = model_longshort.tvalues['mktrf']

    beta_smb_longshort = model_longshort.params['smb']
    t_statistic_smb_longshort = model_longshort.tvalues['smb']

    beta_hml_longshort = model_longshort.params['hml']
    t_statistic_hml_longshort = model_longshort.tvalues['hml']

    excess_returns_longshort = y_longshort
    tracking_error_longshort = excess_returns_longshort.std()
    ir_longshort = alpha_longshort / tracking_error_longshort

    results_ff3.append({
        'Portfolio': 'Long-short Portfolio',
        'Period': 'first half' if period == 0 else 'second half',
        'Alpha (FF3)': alpha_longshort,
        'T-Statistic Alpha': t_statistic_alpha_longshort,
        'Beta Mkt-Rf': beta_mkt_longshort,
        'T-Statistic Mkt-Rf': t_statistic_mkt_longshort,
        'Beta SMB': beta_smb_longshort,
        'T-Statistic SMB': t_statistic_smb_longshort,
        'Beta HML': beta_hml_longshort,
        'T-Statistic HML': t_statistic_hml_longshort,
        'Information Ratio (FF3)': ir_longshort
    })

results_ff3_df = pd.DataFrame(results_ff3)
results_ff3_df

In [ ]:
# Calculating the drawdowns

def calculate_drawdowns(wealth_index):
    previous_peaks = wealth_index.cummax()
    drawdowns = (wealth_index - previous_peaks) / previous_peaks
    return drawdowns

drawdowns_long = calculate_drawdowns(long_portfolio['cumulative_returns']+1)
drawdowns_long_short = calculate_drawdowns(new_long_short['cumulative_returns']+1)

# Plotiing

fig = go.Figure()

fig.add_trace(go.Scatter(x=long_portfolio['datadate'], y=drawdowns_long, mode='lines', name='Long Portfolio'))
fig.add_trace(go.Scatter(x=new_long_short['datadate'], y=drawdowns_long_short, mode='lines', name='Long-Short Portfolio'))

fig.update_layout(
    title='Portfolios Drawdowns',
    xaxis_title='Date',
    yaxis_title='Drawdown',
    paper_bgcolor='white',  
    plot_bgcolor='white',   
    xaxis=dict(linecolor='black', linewidth=1, mirror=True, gridcolor='lightgray', gridwidth=0.5),  
    yaxis=dict(linecolor='black', linewidth=1, mirror=True, gridcolor='lightgray', gridwidth=0.5, zeroline=True, zerolinecolor='lightgray', zerolinewidth=0.5), 
    legend=dict(x=0.01, y=0.01, traceorder='normal', bgcolor='white', bordercolor='black', borderwidth=0.2), 
    height=500, 
    width=1000,   
)

fig.show()

## Analyzing the Dynamic Relationship Between Consumer Spending Patterns and Stock Market Sector Performance

In [ ]:
import os
import requests
import pandas as pd
from zipfile import ZipFile
import io
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import wrds
import statsmodels.api as sm
import plotly.express as px  
import plotly.graph_objects as go

In [ ]:
# Download the files using URLs

urls = ['https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/12_Industry_Portfolios_daily_CSV.zip',
        'https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_daily_CSV.zip',
        'https://www.census.gov/econ_getzippedfile/?programCode=MRTS',
        'https://www.census.gov/retail/marts/www/MARTSreleasedates.xls'
]

local_filenames = ['12_Industry_Portfolios_Daily.zip',
                   'F-F_Research_Data_Factors_daily_CSV.zip',
                   'MRTS-mf.zip',
                   'MARTSreleasedates.xls']

current_directory = os.getcwd()

try:
    for url, local_filename in zip(urls, local_filenames):
        response = requests.get(url)
        if response.status_code == 200:
            local_file_path = os.path.join(current_directory, local_filename)
            with open(local_file_path, 'wb') as file:
                file.write(response.content)
            print(f"File downloaded as {local_file_path}")
        else:
            print(f"Failed to download file from {url}. Status code: {response.status_code}")

except Exception as e:
    print(f"An error occurred: {str(e)}")


In [ ]:
# Getting Sector Daily Returns

zip_file_path = '12_Industry_Portfolios_Daily.zip'


with ZipFile(zip_file_path, 'r') as archive:
    with archive.open('12_Industry_Portfolios_Daily.csv') as csv_file:
        df_Industry_Portfolios = pd.read_csv(io.TextIOWrapper(csv_file),skiprows=9)
        

index_to_cut = df_Industry_Portfolios[df_Industry_Portfolios['Unnamed: 0'] == '  Average Equal Weighted Returns -- Daily'].index[0]
df_Industry_Portfolios = df_Industry_Portfolios.iloc[:index_to_cut]
df_Industry_Portfolios.rename(columns={df_Industry_Portfolios.columns[0]: 'Date'}, inplace=True)
df_Industry_Portfolios['Date'] = pd.to_datetime(df_Industry_Portfolios['Date'], format='%Y%m%d').dt.strftime('%Y-%m-%d')
df_Industry_Portfolios.columns = df_Industry_Portfolios.columns.str.replace(' ', '')
df_Industry_Portfolios = df_Industry_Portfolios[df_Industry_Portfolios['Date'] <= '2023-10-31'].copy()

In [ ]:
# Getting the 452-General Merchandise Stores Signal from MRTS

zip_file_path = 'MRTS-mf.zip'

with ZipFile(zip_file_path, 'r') as archive:
    with archive.open('MRTS-mf.csv') as csv_file:
        df_ConsumerData = pd.read_csv(io.TextIOWrapper(csv_file),skiprows=496)

with ZipFile(zip_file_path, 'r') as archive:
    with archive.open('MRTS-mf.csv') as csv_file:
        df_Categories = pd.read_csv(io.TextIOWrapper(csv_file),skiprows = 1, nrows=65)
        
with ZipFile(zip_file_path, 'r') as archive:
    with archive.open('MRTS-mf.csv') as csv_file:
        df_Time_Periods = pd.read_csv(io.TextIOWrapper(csv_file),skiprows=93, nrows=383)

df_ConsumerData = df_ConsumerData[df_ConsumerData['dt_idx']==5]
df_ConsumerData = df_ConsumerData[df_ConsumerData['et_idx']==0]
df_ConsumerData = df_ConsumerData[df_ConsumerData['is_adj']==1]

mapping_dict_Categories = df_Categories.set_index('cat_idx')['cat_desc'].to_dict()
mapping_dict_Time_Periods = df_Time_Periods.set_index('per_idx')['per_name'].to_dict()

df_ConsumerData['cat_desc'] = df_ConsumerData['cat_idx'].map(mapping_dict_Categories)
df_ConsumerData['per_name'] = df_ConsumerData['per_idx'].map(mapping_dict_Time_Periods)

df_ConsumerData = df_ConsumerData[['per_name','cat_desc','val']]
df_ConsumerData = df_ConsumerData.copy()

df_ConsumerData[['Month', 'Year']] = df_ConsumerData['per_name'].str.split('-', n=1, expand=True)

df_ConsumerData.drop(columns=['per_name'], inplace=True)

month_mapping = {
    'Jan': 'Mar',
    'Feb': 'Apr',
    'Mar': 'May',
    'Apr': 'Jun',
    'May': 'Jul',
    'Jun': 'Aug',
    'Jul': 'Sep',
    'Aug': 'Oct',
    'Sep': 'Nov',
    'Oct': 'Dec',
    'Nov': 'Jan',  
    'Dec': 'Feb'   
}

df_ConsumerData['Month'] = df_ConsumerData['Month'].map(month_mapping)

df_ConsumerData['Year'] = df_ConsumerData['Year'].astype(int)
mask = (df_ConsumerData['Month'].isin(['Jan', 'Feb']))
df_ConsumerData['Year'] = df_ConsumerData['Year'].where(~mask, df_ConsumerData['Year'] + 1)


In [ ]:
# Getting the MRTS release dates

excel_file_path = 'MARTSreleasedates.xls'

df_matrix = pd.read_excel(excel_file_path, header=None, skiprows=4)

df_matrix.rename(columns={df_matrix.columns[0]: 'Year'}, inplace=True)


month_names = [
    'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
    'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'
]


df_matrix.columns = ['Year'] + month_names

df_matrix = df_matrix.iloc[:-11]

df_matrix.replace('14*', '14', inplace=True)
df_matrix.replace('11*', '11', inplace=True)
df_matrix.replace('1*,18*', '1', inplace=True)
df_matrix = df_matrix.fillna(0)
df_matrix = df_matrix.astype(float)
df_matrix = df_matrix[df_matrix['Year'] >= 1992]

df_matrix_tidy = df_matrix.melt(id_vars=['Year'], var_name='Month', value_name='Days')

df_matrix_tidy['Year'] = df_matrix_tidy['Year'].astype(int)
df_matrix_tidy['Month'] = df_matrix_tidy['Month'].astype(str)
df_matrix_tidy['Days'] = df_matrix_tidy['Days'].astype(int)

df_ConsumerData['Year'] = df_ConsumerData['Year'].astype(int)
df_ConsumerData['Month'] = df_ConsumerData['Month'].astype(str)

df_ConsumerData['val'] = pd.to_numeric(df_ConsumerData['val'], errors='coerce')

merged_df = df_ConsumerData.merge(df_matrix_tidy, on=['Year', 'Month'], how='left')

mask_0 = (merged_df['Month'] == 'Dec') & (merged_df['Year'] == 2018) & (merged_df['Days'] == 14)
mask_0 = (merged_df['Month'] == 'Dec') & (merged_df['Year'] == 2018) & (merged_df['Days'] == 14)

mask_1 = (merged_df['Month'] == 'Jan') & (merged_df['Year'] == 2019) & (merged_df['Days'] == 0)
mask_1 = (merged_df['Month'] == 'Jan') & (merged_df['Year'] == 2019) & (merged_df['Days'] == 0)

mask_2 = (merged_df['Month'] == 'Feb') & (merged_df['Year'] == 2019) & (merged_df['Days'] == 14)
mask_2 = (merged_df['Month'] == 'Feb') & (merged_df['Year'] == 2019) & (merged_df['Days'] == 14)

mask_3 = (merged_df['Month'] == 'Mar') & (merged_df['Year'] == 2019) & (merged_df['Days'] == 11)
mask_3 = (merged_df['Month'] == 'Mar') & (merged_df['Year'] == 2019) & (merged_df['Days'] == 11)

merged_df.loc[mask_0, 'Month'] = 'Feb'
merged_df.loc[mask_0, 'Year'] = 2019
merged_df.loc[mask_0, 'Days'] = 14

merged_df.loc[mask_1, 'Month'] = 'Mar'
merged_df.loc[mask_1, 'Days'] = 11

merged_df.loc[mask_2, 'Month'] = 'Apr'
merged_df.loc[mask_2, 'Days'] = 1

merged_df.loc[mask_3, 'Month'] = 'Apr'
merged_df.loc[mask_3, 'Days'] = 18

merged_df['Date'] = pd.to_datetime(merged_df['Year'].astype(str) + '-' + merged_df['Month'] + '-' + merged_df['Days'].astype(str), format='%Y-%b-%d')
merged_df = merged_df[['Date','cat_desc','val']]

merged_df['signal'] = 0

# Generating the signal using 452 direction
prev_signals = {}

for index, row in merged_df.iterrows():
    cat_desc = row['cat_desc']
    val = row['val']

    if val > 0:
        merged_df.at[index, 'signal'] = 1
        prev_signals[cat_desc] = 1
    elif val < 0:
        merged_df.at[index, 'signal'] = -1
        prev_signals[cat_desc] = -1
    elif val == 0 and cat_desc in prev_signals:
        merged_df.at[index, 'signal'] = prev_signals[cat_desc]

merged_df.sort_values(by = 'Date')
merged_df.reset_index(drop=True, inplace=True)

categories_to_keep = ['452: General Merchandise Stores']
merged_df = merged_df[merged_df['cat_desc'].str.startswith(tuple(categories_to_keep))]

In [ ]:
# Creating the returns based on signals
signals_df = merged_df.copy()

signals_df = signals_df.groupby(['Date', 'cat_desc']).agg({'signal': 'mean'}).reset_index()

signals_df = signals_df.pivot(index='Date', columns='cat_desc', values='signal')

signals_df.reset_index(inplace=True)

df_Industry_Portfolios = df_Industry_Portfolios.copy()

signals_df['Date'] = pd.to_datetime(signals_df['Date'])
df_Industry_Portfolios['Date'] = pd.to_datetime(df_Industry_Portfolios['Date'])

combined_df = df_Industry_Portfolios.merge(signals_df, on='Date', how='left')

combined_df.fillna(method='ffill', inplace=True)
combined_df = combined_df[combined_df['Date'] >= '1992-04-14'].copy()
combined_df.reset_index(inplace=True)
del combined_df['index']

data = combined_df

df_base = pd.DataFrame(data)
df_base['Date'] = pd.to_datetime(df_base['Date'])
df_base.set_index('Date', inplace=True)

columns_for_returns = [
    'NoDur', 'Durbl', 'Manuf', 'Enrgy', 'Chems', 'BusEq', 'Telcm', 'Utils', 'Shops',
    'Hlth', 'Money', 'Other'
]

signal_column = '452: General Merchandise Stores'

for return_column in columns_for_returns:
    df_base[return_column] = pd.to_numeric(df_base[return_column], errors='coerce')
    df_base[return_column + '_Port'] = (
        df_base[return_column] * df_base[signal_column] 
    )

In [ ]:
# FF3 factors

zip_file_path = 'F-F_Research_Data_Factors_daily_CSV.zip'

with ZipFile(zip_file_path, 'r') as archive:
    with archive.open('F-F_Research_Data_Factors_daily.CSV') as csv_file:
        ff3_df = pd.read_csv(io.TextIOWrapper(csv_file),skiprows=3)


index_to_cut = ff3_df[ff3_df['Unnamed: 0'] == 'Copyright 2023 Kenneth R. French'].index[0]
ff3_df = ff3_df.iloc[:index_to_cut]
ff3_df = ff3_df.rename(columns={'Unnamed: 0': 'Date'})
ff3_df['Date'] = pd.to_datetime(ff3_df['Date'], format='%Y%m%d')
ff3_df = ff3_df[ff3_df['Date'] >= '1992-04-14'].copy()
ff3_df.set_index('Date', inplace=True)
df_base = df_base.merge(ff3_df, on='Date', how='left')

In [ ]:
# Performance of the sectors

portfolio_names = df_base.columns[13:25]
portfolio_returns = df_base.iloc[:,13:25]
portfolio_returns = portfolio_returns/100.0

performance_stats = pd.DataFrame(index=portfolio_names)

for portfolio_name, returns in portfolio_returns.items():
    alpha = 0.01 
    var = np.percentile(returns, alpha*100)

    cumulative_returns = (1 + returns).cumprod()
    cumulative_max = cumulative_returns.cummax() 
    drawdown = cumulative_returns / cumulative_max - 1
    max_drawdown = drawdown.min()
    
    max_cum_return = cumulative_max.max() - 1

    drawdown = pd.to_numeric(drawdown, errors='coerce')
    worst_periods = drawdown.nsmallest(1)
    worst_period_dates = returns.index[drawdown.index.isin(worst_periods.index)]
    
    average_returns = returns.mean()*252
    average_vola = (returns.std() * np.sqrt(252))
    
    cum_return = (1 + returns).prod() - 1

    skewness = returns.skew()
    kurt = returns.kurtosis()

    sharpe_ratio = average_returns/ average_vola

    positive_days = returns[returns > 0] 
    percentage_positive_days = (len(positive_days) / len(returns)) * 100

    annual_returns = returns.resample('Y').apply(lambda x: (1 + x).prod() - 1)

    max_annual_return = annual_returns.max()
    min_annual_return = annual_returns.min()
    best_year = annual_returns.idxmax()
    worst_year = annual_returns.idxmin()

    performance_stats.at[portfolio_name, 'Annualized Return'] = "{:.2f}%".format(average_returns*100)
    performance_stats.at[portfolio_name, 'Annualized Volatility'] = "{:.2f}%".format(average_vola*100)
    performance_stats.at[portfolio_name, 'Sharpe Ratio'] = "{:.3f}".format(sharpe_ratio)
    performance_stats.at[portfolio_name, 'Max Annual Return'] = "{:.2f}%".format(max_annual_return * 100)
    performance_stats.at[portfolio_name, 'Best Year'] = best_year.strftime('%Y')
    performance_stats.at[portfolio_name, 'Min Annual Return'] = "{:.2f}%".format(min_annual_return * 100)
    performance_stats.at[portfolio_name, 'Worst Year'] = worst_year.strftime('%Y')
    performance_stats.at[portfolio_name, 'VaR'] = "{:.2f}%".format(var * 100)
    performance_stats.at[portfolio_name, 'Skew'] = "{:.3f}".format(skewness)
    performance_stats.at[portfolio_name, 'Kurtosis'] = "{:.3f}".format(kurt)
    performance_stats.at[portfolio_name, 'Cumulative Return'] = "{:.2f}%".format(cum_return*100)
    performance_stats.at[portfolio_name, 'Max Cum. Return'] = "{:.2f}%".format(max_cum_return*100)
    performance_stats.at[portfolio_name, 'Max Drawdown'] = "{:.2f}%".format(max_drawdown * 100) 
    performance_stats.at[portfolio_name, 'Worst Perf. Period'] = worst_period_dates.to_list()
    performance_stats.at[portfolio_name, 'Positive Days'] = "{:.2f}%".format(percentage_positive_days)

performance_stats


In [ ]:
# Plotting cumulative retusn of the sectors
columns_for_cumprod = [return_column + '_Port' for return_column in columns_for_returns]

df_base_cum = pd.DataFrame(df_base)
df_base_cum[columns_for_cumprod] = (1 + df_base_cum[columns_for_cumprod]/100.0).cumprod() - 1

sorted_columns = df_base_cum[columns_for_cumprod].iloc[-1].sort_values(ascending=False).index
df_base_sorted = df_base_cum[columns_for_cumprod][sorted_columns]

fig = px.line(df_base_sorted, y=df_base_sorted.columns, title='Cumulative Returns')

fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Portfolio Cumulative Return',
    xaxis=dict(title='Date', showline=True, showgrid=True, gridwidth=0.2, gridcolor='lightgray', showticklabels=True, linecolor='black'),
    yaxis=dict(title='Cumulative Return', showline=True, showgrid=True, gridwidth=0.2, gridcolor='lightgray', showticklabels=True, linecolor='black'),
    legend=dict(
        orientation="h",
        x=0.5,
        y=-0.15,
        xanchor='center',
        yanchor='top',
        bgcolor='rgba(0,0,0,0)'),
)

fig.add_shape(
    type="rect",
    x0=0,
    x1=1,
    y0=0,
    y1=1,
    xref="paper",
    yref="paper",
    line=dict(color="black", width=1.5),
    layer="below"
)

fig.update_layout(
    plot_bgcolor='white',
    paper_bgcolor='white',
    legend_title='',
)

fig.show()

In [ ]:
# CAPM and FF3 for the sectors, full period

market_df = ff3_df.merge(df_base.iloc[:, 13:25], left_index=True, right_index=True, how='left')

portfolio_results = []

for portfolio_return in market_df.columns[4:]:
    try:
        #CAPM
        market_return = market_df['Mkt-RF']
        excess_return = market_df[portfolio_return]

        results_capm = sm.OLS(excess_return, sm.add_constant(market_return)).fit()

        alpha_capm = results_capm.params[0]
        t_statistic_alpha_capm = results_capm.tvalues[0]

        tracking_error_capm = excess_return.std()
        ir_capm = alpha_capm / tracking_error_capm
        
        beta_mkt_capm = results_capm.params[1]
        t_statistic_beta_mkt_capm = results_capm.tvalues[1]
        
        #FF3
        market_return = market_df[['Mkt-RF', 'SMB', 'HML']]
        excess_return = market_df[portfolio_return]

        results_ff3 = sm.OLS(excess_return, sm.add_constant(market_return)).fit()

        alpha_ff3 = results_ff3.params[0]
        t_statistic_alpha_ff3 = results_ff3.tvalues[0]
        
        tracking_error_ff3 = excess_return.std()
        ir_ff3 = alpha_ff3 / tracking_error_ff3
        
        beta_mkt_ff3 = results_ff3.params[1]
        beta_smb_ff3 = results_ff3.params[2]
        beta_hml_ff3 = results_ff3.params[3]

        
        t_statistic_beta_mkt_ff3 = results_ff3.tvalues[1]
        t_statistic_beta_smb_ff3 = results_ff3.tvalues[2]
        t_statistic_beta_hml_ff3 = results_ff3.tvalues[3]

        result_dict = {'Portfolio': portfolio_return,
                       'IR CAPM': ir_capm,
                       'Alpha ': alpha_capm,
                       'T-Stat ': t_statistic_alpha_capm,
                       
                       'Beta Mkt-Rf': beta_mkt_capm,
                       'T-Stat Beta Mkt-Rf': t_statistic_beta_mkt_capm,
                       
                       'IR FF3': ir_ff3,
                       'Alpha': alpha_ff3,
                       'T-Stat': t_statistic_alpha_ff3,
        
                       'Beta MKT': beta_mkt_ff3,
                       'T-Stat Beta MKT': t_statistic_beta_mkt_ff3,
                       'Beta SMB': beta_smb_ff3,
                       'T-Stat Beta SMB': t_statistic_beta_smb_ff3,
                       'Beta HML': beta_hml_ff3,
                       'T-Stat Beta HML': t_statistic_beta_hml_ff3,
                      }

        portfolio_results.append(result_dict)
        
    except Exception as e:
        print(f"Error occurred: {e}")
results_df = pd.DataFrame(portfolio_results)
results_df


## Period Analysis 

In [ ]:
# Defining the date ranges for the two periods and their performance
periods = [
    ("Period 1", pd.to_datetime("1992-04-14"), pd.to_datetime("2007-12-31")),
    ("Period 2", pd.to_datetime("2008-01-01"), pd.to_datetime("2023-12-31"))
]

period_stats = {period_name: pd.DataFrame(index=portfolio_names) for period_name, _, _ in periods}

for portfolio_name, returns in portfolio_returns.items():
    for period_name, period_start, period_end in periods:
        returns_period = returns.loc[(returns.index >= period_start) & (returns.index <= period_end)]
        
        alpha = 0.01
        var_period = np.percentile(returns_period, alpha * 100)

        cumulative_returns_period = (1 + returns_period).cumprod()
        cumulative_max_period = cumulative_returns_period.cummax()
        drawdown_period = cumulative_returns_period / cumulative_max_period - 1
        max_drawdown_period = drawdown_period.min()

        max_cum_return_period = cumulative_max_period.max() - 1
        
        drawdown_period = pd.to_numeric(drawdown_period, errors='coerce')

        worst_periods_period = drawdown_period.nsmallest(1).index
        worst_period_dates_period = returns_period.index[drawdown_period.index.isin(worst_periods_period)]

        average_returns_period = returns_period.mean()*252
        average_vola_period = (returns_period.std() * np.sqrt(252))

        cum_return_period = (1 + returns_period).prod() - 1

        skewness_period = returns_period.skew()
        kurtosis_period = returns_period.kurtosis()

        sharpe_ratio_period = average_returns_period / average_vola_period

        positive_days_period = returns_period[returns_period > 0] 
        percentage_positive_days_period = (len(positive_days_period) / len(returns_period)) * 100

        annual_returns = returns_period.resample('Y').apply(lambda x: (1 + x).prod() - 1)

        max_annual_return = annual_returns.max()
        min_annual_return = annual_returns.min()
        best_year = annual_returns.idxmax()
        worst_year = annual_returns.idxmin()

        period_stats[period_name].at[portfolio_name, 'Annualized Return'] = "{:.2f}%".format(average_returns_period * 100)
        period_stats[period_name].at[portfolio_name, 'Annualized Volatility'] = "{:.2f}%".format(average_vola_period * 100)
        period_stats[period_name].at[portfolio_name, 'Sharpe Ratio'] = "{:.3f}".format(sharpe_ratio_period)
        period_stats[period_name].at[portfolio_name, 'Max Annual Return'] = "{:.2f}%".format(max_annual_return * 100)
        period_stats[period_name].at[portfolio_name, 'Best Year'] = best_year.strftime('%Y')
        period_stats[period_name].at[portfolio_name, 'Min Annual Return'] = "{:.2f}%".format(min_annual_return * 100)
        period_stats[period_name].at[portfolio_name, 'Worst Year'] = worst_year.strftime('%Y')
        period_stats[period_name].at[portfolio_name, 'VaR'] = "{:.2f}%".format(var_period * 100)
        period_stats[period_name].at[portfolio_name, 'Skew'] = "{:.3f}".format(skewness_period)
        period_stats[period_name].at[portfolio_name, 'Kurtosis'] = "{:.3f}".format(kurtosis_period)
        period_stats[period_name].at[portfolio_name, 'Cumulative Return'] = "{:.2f}%".format(cum_return_period * 100)
        period_stats[period_name].at[portfolio_name, 'Max Cum. Return'] = "{:.2f}%".format(max_cum_return_period * 100)
        period_stats[period_name].at[portfolio_name, 'Max Drawdown'] = "{:.2f}%".format(max_drawdown_period * 100)
        period_stats[period_name].at[portfolio_name, 'Worst Perf. Period'] = worst_period_dates_period.to_list()
        period_stats[period_name].at[portfolio_name, 'Positive Days'] = "{:.2f}%".format(percentage_positive_days_period)

In [ ]:
# CAPM and FF3 in the periods
def calculate_metrics_capm(portfolio_returns, market_returns):
    X = sm.add_constant(market_returns)
    model = sm.OLS(portfolio_returns, X).fit()
    
    alpha_capm = model.params[0]
    beta_mkt_capm = model.params[1]
    t_statistic_alpha_capm = model.tvalues[0]
    t_statistic_beta_mkt_capm = model.tvalues[1]
    
    tracking_error_capm = portfolio_returns.std()
    ir_capm = alpha_capm / tracking_error_capm
        
    return pd.Series({
        'IR CAPM': ir_capm,
        'Alpha CAPM': alpha_capm,
        'T-Stat CAPM': t_statistic_alpha_capm,               
        'Beta Mkt-Rf': beta_mkt_capm,
        'T-Stat Beta Mkt-Rf': t_statistic_beta_mkt_capm, 
        })

def calculate_metrics_ff3(portfolio_returns, market_returns):
    X = sm.add_constant(market_returns)
    model = sm.OLS(portfolio_returns, X).fit()
    
    alpha_ff5 = model.params[0]
    t_statistic_alpha_ff5 = model.tvalues[0]
    
    tracking_error_ff5 = portfolio_returns.std()
    ir_ff5 = alpha_ff5 / tracking_error_ff5
    
    beta_mkt_ff5 = model.params[1]
    beta_smb_ff5 = model.params[2]
    beta_hml_ff5 = model.params[3]

    
    t_statistic_beta_mkt_ff5 = model.tvalues[1]
    t_statistic_beta_smb_ff5 = model.tvalues[2]
    t_statistic_beta_hml_ff5 = model.tvalues[3]

        
    return pd.Series({
        'IR FF3': ir_ff5,
        'Alpha FF3': alpha_ff5,
        'T-Stat FF3': t_statistic_alpha_ff5,               
        'Beta MKT': beta_mkt_ff5,
        'T-Stat Beta MKT': t_statistic_beta_mkt_ff5,
        'Beta SMB': beta_smb_ff5,
        'T-Stat Beta SMB': t_statistic_beta_smb_ff5,
        'Beta HML': beta_hml_ff5,
        'T-Stat Beta HML': t_statistic_beta_hml_ff5,

        })

portfolio_columns = portfolio_returns.columns

results_period1 = pd.DataFrame(index=portfolio_columns)
results_period2 = pd.DataFrame(index=portfolio_columns)

for portfolio_column in portfolio_columns:
    for period, start_date, end_date in periods:
        
        portfolio_returns_period = portfolio_returns.loc[start_date:end_date, portfolio_column].reset_index(drop=True)
        
        market_returns_period_capm = ff3_df.loc[start_date:end_date, 'Mkt-RF'].reset_index(drop=True)
        market_returns_period_ff5 = ff3_df.loc[start_date:end_date, ['Mkt-RF', 'SMB', 'HML']].reset_index(drop=True)
        
        metrics_capm = calculate_metrics_capm(portfolio_returns_period, market_returns_period_capm)
        metrics_ff3 = calculate_metrics_ff3(portfolio_returns_period, market_returns_period_ff5)
        
        if period == "Period 1":
            results_df = results_period1
        elif period == "Period 2":
            results_df = results_period2
        else:
            raise ValueError(f"Unknown period: {period}")
        
        
        results_df.loc[portfolio_column, 'CAPM_IR'] = metrics_capm['IR CAPM']
        results_df.loc[portfolio_column, 'Alpha '] = metrics_capm['Alpha CAPM']
        results_df.loc[portfolio_column, 'T-Stat '] = metrics_capm['T-Stat CAPM']
        results_df.loc[portfolio_column, 'Beta_Mkt-Rf'] = metrics_capm['Beta Mkt-Rf']
        results_df.loc[portfolio_column, 'T-Stat Beta_Mkt-Rf'] = metrics_capm['T-Stat Beta Mkt-Rf']

        results_df.loc[portfolio_column, 'FF3_IR'] = metrics_ff3['IR FF3']
        results_df.loc[portfolio_column, 'Alpha'] = metrics_ff3['Alpha FF3']
        results_df.loc[portfolio_column, 'T-Stat'] = metrics_ff3['T-Stat FF3']
        results_df.loc[portfolio_column, 'Beta_MKT'] = metrics_ff3['Beta MKT']
        results_df.loc[portfolio_column, 'T-Stat Beta_MKT'] = metrics_ff3['T-Stat Beta MKT']
        results_df.loc[portfolio_column, 'Beta_SMB'] = metrics_ff3['Beta SMB']
        results_df.loc[portfolio_column, 'T-Stat Beta_SMB'] = metrics_ff3['T-Stat Beta SMB']
        results_df.loc[portfolio_column, 'Beta_HML'] = metrics_ff3['Beta HML']
        results_df.loc[portfolio_column, 'T-Stat Beta_HML'] = metrics_ff3['T-Stat Beta HML']


In [ ]:
period_stats["Period 1"]

In [ ]:
results_period1

In [ ]:
# Plotting period cumulative returns
cumulative_returns = {str(period_name): {} for period_name, _, _ in periods}

for portfolio_name, returns in portfolio_returns.items():
    for period_name, period_start, period_end in periods:
        returns_period = returns.loc[(returns.index >= period_start) & (returns.index <= period_end)]

        cumulative_returns_period = (1 + returns_period).cumprod() - 1
        key = str(period_name)  
        cumulative_returns[key][portfolio_name] = cumulative_returns_period

period_name = periods[0][0]
key = str(period_name)

sorted_portfolio_names = sorted(
    cumulative_returns[key].keys(),
    key=lambda portfolio_name: cumulative_returns[key][portfolio_name].iloc[-1],
    reverse=True
)

traces = []
for portfolio_name in sorted_portfolio_names:
    cumulative_return = cumulative_returns[key][portfolio_name]
    trace = go.Scatter(x=cumulative_return.index, y=cumulative_return, mode='lines', name=portfolio_name)
    traces.append(trace)

layout = go.Layout(
    title=f'{period_name} Cumulative Returns',
    xaxis=dict(title='Date', showline=True, showgrid=True, gridwidth=0.2, gridcolor='lightgray', showticklabels=True, linecolor='black'),
    yaxis=dict(title='Cumulative Return', showline=True, showgrid=True, gridwidth=0.2, gridcolor='lightgray', showticklabels=True, linecolor='black'),
    legend=dict(
        orientation="h",
        x=0.5,
        y=-0.15,
        xanchor='center',
        yanchor='top',
        bgcolor='rgba(0,0,0,0)'),
)

fig = go.Figure(data=traces, layout=layout)

fig.add_shape(
    type="rect",
    x0=0,
    x1=1,
    y0=0,
    y1=1,
    xref="paper",
    yref="paper",
    line=dict(color="black", width=1.5),
    layer="below"
)

fig.update_layout(
    plot_bgcolor='white',
    paper_bgcolor='white',
    legend_title='',
)
fig.show()

In [ ]:
period_stats["Period 2"]

In [ ]:
results_period2

In [ ]:
period_name = periods[1][0]
key = str(period_name)

sorted_portfolio_names = sorted(
    cumulative_returns[key].keys(),
    key=lambda portfolio_name: cumulative_returns[key][portfolio_name].iloc[-1],
    reverse=True
)

traces = []
for portfolio_name in sorted_portfolio_names:
    cumulative_return = cumulative_returns[key][portfolio_name]
    trace = go.Scatter(x=cumulative_return.index, y=cumulative_return, mode='lines', name=portfolio_name)
    traces.append(trace)

layout = go.Layout(
    title=f'{period_name} Cumulative Returns',
    xaxis=dict(title='Date', showline=True, showgrid=True, gridwidth=0.2, gridcolor='lightgray', showticklabels=True, linecolor='black'),
    yaxis=dict(title='Cumulative Return', showline=True, showgrid=True, gridwidth=0.2, gridcolor='lightgray', showticklabels=True, linecolor='black'),
    legend=dict(
        orientation="h",
        x=0.5,
        y=-0.15,
        xanchor='center',
        yanchor='top',
        bgcolor='rgba(0,0,0,0)'),
)

fig = go.Figure(data=traces, layout=layout)

fig.add_shape(
    type="rect",
    x0=0,
    x1=1,
    y0=0,
    y1=1,
    xref="paper",
    yref="paper",
    line=dict(color="black", width=1.5),
    layer="below"
)

fig.update_layout(
    plot_bgcolor='white',
    paper_bgcolor='white',
    legend_title='',
)

fig.show()

## Long-short Portfolio

In [ ]:
# Creating the Long-Only and Long-Short Portfolios

portfolio_returns_longShort = pd.DataFrame(index=portfolio_returns.index, columns=['LongShortPortfolio'])
portfolio_returns_longShort['LongShortPortfolio'] = (1/12) * portfolio_returns.sum(axis=1)

raw_return= pd.DataFrame(index=portfolio_returns.index)
raw_return = df_base.iloc[:,:12]/100
raw_return = raw_return[raw_return.index >= '1992-04-14'].copy()
portfolio_returns_longOnly = pd.DataFrame(index=raw_return.index, columns=['LongOnlyPortfolio'])

portfolio_returns_longOnly['LongOnlyPortfolio'] = (1/12) * raw_return.sum(axis=1)

market = pd.DataFrame(index=raw_return.index, columns=['Market'])
market = market[market.index >= '1992-04-14'].copy()
market['Market'] = ff3_df['Mkt-RF']/100

all_portfolios = pd.concat([portfolio_returns_longShort, portfolio_returns_longOnly, market], axis=1)
all_portfolios_cum = (1+all_portfolios).cumprod()-1


# Plotting
fig = px.line(all_portfolios_cum, y=all_portfolios_cum.columns, title='Cumulative Returns')

fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Portfolio Cumulative Return',
    xaxis=dict(showline=True, showgrid=True, gridwidth=0.2, gridcolor='lightgray', showticklabels=True, linecolor='black'),
    yaxis=dict(showline=True, showgrid=True, gridwidth=0.2, gridcolor='lightgray', showticklabels=True, linecolor='black'),
    plot_bgcolor='white',
    paper_bgcolor='white',
    legend_title='',
    legend=dict(
        orientation="h",
        x=0.5,
        y=-0.15,
        xanchor='center',
        yanchor='top',
        bgcolor='rgba(0,0,0,0)'),
)

fig.add_shape(
    type="rect",
    x0=0,
    x1=1,
    y0=0,
    y1=1,
    xref="paper",
    yref="paper",
    line=dict(color="black", width=1.5),
    layer="below"
)

fig.update_layout(width=1000, height=600)
fig.show()


In [ ]:
# Performance Stats for each Portfolio
strat_names = all_portfolios.columns
strat_returns = all_portfolios.iloc[:,:]

strat_stats = pd.DataFrame(index=strat_names)

for strat_name, returns in strat_returns.items():
    alpha = 0.01
    var = np.percentile(returns, alpha*100)

    cumulative_returns = (1 + returns).cumprod()
    cumulative_max = cumulative_returns.cummax() 
    drawdown = cumulative_returns / cumulative_max - 1
    max_drawdown = drawdown.min()
    
    max_cum_return = cumulative_max.max() - 1

    drawdown = pd.to_numeric(drawdown, errors='coerce')
    worst_periods = drawdown.nsmallest(1)
    worst_period_dates = returns.index[drawdown.index.isin(worst_periods.index)]
    
    average_returns = returns.mean()*252
    average_vola = (returns.std() * np.sqrt(252))
    
    cum_return = (1 + returns).prod() - 1

    skewness = returns.skew()
    kurt = returns.kurtosis()

    sharpe_ratio = average_returns/ average_vola

    positive_days = returns[returns > 0]
    percentage_positive_days = (len(positive_days) / len(returns)) * 100
    
    annual_returns = returns.resample('Y').apply(lambda x: (1 + x).prod() - 1)

    max_annual_return = annual_returns.max()
    min_annual_return = annual_returns.min()
    best_year = annual_returns.idxmax()
    worst_year = annual_returns.idxmin()

    strat_stats.at[strat_name, 'Annualized Return'] = "{:.2f}%".format(average_returns*100)
    strat_stats.at[strat_name, 'Annualized Volatility'] = "{:.2f}%".format(average_vola*100)
    strat_stats.at[strat_name, 'Sharpe Ratio'] = "{:.3f}".format(sharpe_ratio)
    strat_stats.at[strat_name, 'Max Annual Return'] = "{:.2f}%".format(max_annual_return * 100)
    strat_stats.at[strat_name, 'Best Year'] = best_year.strftime('%Y')
    strat_stats.at[strat_name, 'Min Annual Return'] = "{:.2f}%".format(min_annual_return * 100)
    strat_stats.at[strat_name, 'Worst Year'] = worst_year.strftime('%Y')
    strat_stats.at[strat_name, 'VaR'] = "{:.2f}%".format(var * 100)
    strat_stats.at[strat_name, 'Skew'] = "{:.3f}".format(skewness)
    strat_stats.at[strat_name, 'Kurtosis'] = "{:.3f}".format(kurt)
    strat_stats.at[strat_name, 'Cumulative Return'] = "{:.2f}%".format(cum_return*100)
    strat_stats.at[strat_name, 'Max Cum. Return'] = "{:.2f}%".format(max_cum_return*100)
    strat_stats.at[strat_name, 'Max Drawdown'] = "{:.2f}%".format(max_drawdown * 100) 
    strat_stats.at[strat_name, 'Worst Perf. Period'] = worst_period_dates.to_list()
    strat_stats.at[strat_name, 'Positive Days'] = "{:.2f}%".format(percentage_positive_days)

strat_stats

In [ ]:
# CAPM and FF3 for the 2 portfolios
market_df = ff3_df.merge(all_portfolios, left_index=True, right_index=True, how='left')

portfolio_results = []

for portfolio_return in market_df.columns[4:6]:
    try:
        #CAPM
        market_return = market_df['Mkt-RF']
        excess_return = market_df[portfolio_return]

        results_capm = sm.OLS(excess_return, sm.add_constant(market_return)).fit()

        alpha_capm = results_capm.params[0]
        t_statistic_alpha_capm = results_capm.tvalues[0]

        tracking_error_capm = excess_return.std()
        ir_capm = alpha_capm / tracking_error_capm
        
        beta_mkt_capm = results_capm.params[1]
        t_statistic_beta_mkt_capm = results_capm.tvalues[1]
        
        #FF3
        market_return = market_df[['Mkt-RF', 'SMB', 'HML']]
        excess_return = market_df[portfolio_return]

        results_ff5 = sm.OLS(excess_return, sm.add_constant(market_return)).fit()

        alpha_ff5 = results_ff5.params[0]
        t_statistic_alpha_ff5 = results_ff5.tvalues[0]
        
        tracking_error_ff5 = excess_return.std()
        ir_ff5 = alpha_ff5 / tracking_error_ff5
        
        beta_mkt_ff5 = results_ff5.params[1]
        beta_smb_ff5 = results_ff5.params[2]
        beta_hml_ff5 = results_ff5.params[3]

        
        t_statistic_beta_mkt_ff5 = results_ff5.tvalues[1]
        t_statistic_beta_smb_ff5 = results_ff5.tvalues[2]
        t_statistic_beta_hml_ff5 = results_ff5.tvalues[3]

        result_dict = {'Portfolio': portfolio_return,
                       'IR CAPM': ir_capm,
                       'Alpha ': alpha_capm,
                       'T-Stat ': t_statistic_alpha_capm,
                       
                       'Beta Mkt-Rf': beta_mkt_capm,
                       'T-Stat Beta Mkt-Rf': t_statistic_beta_mkt_capm,
                       
                       'IR FF3': ir_ff5,
                       'Alpha': alpha_ff5,
                       'T-Stat': t_statistic_alpha_ff5,
        
                       'Beta MKT': beta_mkt_ff5,
                       'T-Stat Beta MKT': t_statistic_beta_mkt_ff5,
                       'Beta SMB': beta_smb_ff5,
                       'T-Stat Beta SMB': t_statistic_beta_smb_ff5,
                       'Beta HML': beta_hml_ff5,
                       'T-Stat Beta HML': t_statistic_beta_hml_ff5,

                      }

        portfolio_results.append(result_dict)
        
    except Exception as e:
        print(f"Error occurred: {e}")

results_df = pd.DataFrame(portfolio_results)
results_df

## Portfolio Period Analysis

In [ ]:
# Creating the period for the portfolios and the performance

strat_stats = {period_name: pd.DataFrame(index=strat_names) for period_name, _, _ in periods}

for strat_name, returns in strat_returns.items():
    for period_name, period_start, period_end in periods:
        returns_period = returns.loc[(returns.index >= period_start) & (returns.index <= period_end)]
        alpha = 0.01  
        var_period = np.percentile(returns_period, alpha * 100)

        cumulative_returns_period = (1 + returns_period).cumprod()
        cumulative_max_period = cumulative_returns_period.cummax()
        drawdown_period = cumulative_returns_period / cumulative_max_period - 1
        max_drawdown_period = drawdown_period.min()

        max_cum_return_period = cumulative_max_period.max() - 1
        
        drawdown_period = pd.to_numeric(drawdown_period, errors='coerce')

        worst_periods_period = drawdown_period.nsmallest(1).index
        worst_period_dates_period = returns_period.index[drawdown_period.index.isin(worst_periods_period)]

        average_returns_period = returns_period.mean()*252
        average_vola_period = (returns_period.std() * np.sqrt(252))

        cum_return_period = (1 + returns_period).prod() - 1

        skewness_period = returns_period.skew()
        kurtosis_period = returns_period.kurtosis()

        sharpe_ratio_period = average_returns_period / average_vola_period

        positive_days_period = returns_period[returns_period > 0]
        percentage_positive_days_period = (len(positive_days_period) / len(returns_period)) * 100

        annual_returns = returns_period.resample('Y').apply(lambda x: (1 + x).prod() - 1)

        max_annual_return = annual_returns.max()
        min_annual_return = annual_returns.min()
        best_year = annual_returns.idxmax()
        worst_year = annual_returns.idxmin()

        strat_stats[period_name].at[strat_name, 'Annualized Return'] = "{:.2f}%".format(average_returns_period * 100)
        strat_stats[period_name].at[strat_name, 'Annualized Volatility'] = "{:.2f}%".format(average_vola_period * 100)
        strat_stats[period_name].at[strat_name, 'Sharpe Ratio'] = "{:.3f}".format(sharpe_ratio_period)
        strat_stats[period_name].at[strat_name, 'Max Annual Return'] = "{:.2f}%".format(max_annual_return * 100)
        strat_stats[period_name].at[strat_name, 'Best Year'] = best_year.strftime('%Y')
        strat_stats[period_name].at[strat_name, 'Min Annual Return'] = "{:.2f}%".format(min_annual_return * 100)
        strat_stats[period_name].at[strat_name, 'Worst Year'] = worst_year.strftime('%Y')
        strat_stats[period_name].at[strat_name, 'VaR'] = "{:.2f}%".format(var_period * 100)
        strat_stats[period_name].at[strat_name, 'Skew'] = "{:.3f}".format(skewness_period)
        strat_stats[period_name].at[strat_name, 'Kurtosis'] = "{:.3f}".format(kurtosis_period)
        strat_stats[period_name].at[strat_name, 'Cumulative Return'] = "{:.2f}%".format(cum_return_period * 100)
        strat_stats[period_name].at[strat_name, 'Max Cum. Return'] = "{:.2f}%".format(max_cum_return_period * 100)
        strat_stats[period_name].at[strat_name, 'Max Drawdown'] = "{:.2f}%".format(max_drawdown_period * 100)
        strat_stats[period_name].at[strat_name, 'Worst Perf. Period'] = worst_period_dates_period.to_list()
        strat_stats[period_name].at[strat_name, 'Positive Days'] = "{:.2f}%".format(percentage_positive_days_period)

In [ ]:
# CAPM and FF3
strategy_columns = ['LongShortPortfolio', 'LongOnlyPortfolio']

results_period1 = pd.DataFrame(index=strategy_columns)
results_period2 = pd.DataFrame(index=strategy_columns)

for strategy_column in strategy_columns:
    for period, start_date, end_date in periods:
        
        strategy_returns_period = strat_returns.loc[start_date:end_date, strategy_column].reset_index(drop=True)
        
        market_returns_period_capm = ff3_df.loc[start_date:end_date, 'Mkt-RF'].reset_index(drop=True)
        market_returns_period_ff3 = ff3_df.loc[start_date:end_date, ['Mkt-RF', 'SMB', 'HML']].reset_index(drop=True)
        
        metrics_capm = calculate_metrics_capm(strategy_returns_period, market_returns_period_capm)
        metrics_ff3 = calculate_metrics_ff3(strategy_returns_period, market_returns_period_ff3)
            
        if period == "Period 1":
            results_df = results_period1
        elif period == "Period 2":
            results_df = results_period2
        else:
            raise ValueError(f"Unknown period: {period}")
        
        results_df.loc[strategy_column, 'CAPM_IR'] = metrics_capm['IR CAPM']
        results_df.loc[strategy_column, 'Alpha '] = metrics_capm['Alpha CAPM']
        results_df.loc[strategy_column, 'T-Stat '] = metrics_capm['T-Stat CAPM']
        results_df.loc[strategy_column, 'Beta_Mkt-Rf'] = metrics_capm['Beta Mkt-Rf']        
        results_df.loc[strategy_column, 'T-Stat Beta_Mkt-Rf'] = metrics_capm['T-Stat Beta Mkt-Rf']

        results_df.loc[strategy_column, 'FF3_IR'] = metrics_ff3['IR FF3']
        results_df.loc[strategy_column, 'Alpha'] = metrics_ff3['Alpha FF3']
        results_df.loc[strategy_column, 'T-Stat'] = metrics_ff3['T-Stat FF3']
        results_df.loc[strategy_column, 'Beta_MKT'] = metrics_ff3['Beta MKT']
        results_df.loc[strategy_column, 'T-Stat Beta_MKT'] = metrics_ff3['T-Stat Beta MKT']
        results_df.loc[strategy_column, 'Beta_SMB'] = metrics_ff3['Beta SMB']
        results_df.loc[strategy_column, 'T-Stat Beta_SMB'] = metrics_ff3['T-Stat Beta SMB']
        results_df.loc[strategy_column, 'Beta_HML'] = metrics_ff3['Beta HML']
        results_df.loc[strategy_column, 'T-Stat Beta_HML'] = metrics_ff3['T-Stat Beta HML']



In [ ]:
strat_stats["Period 1"]

In [ ]:
results_period1

In [ ]:
# Plotting
cumulative_returns = {str(period_name): {} for period_name, _, _ in periods}

for strat_name, returns in strat_returns.items():
    for period_name, period_start, period_end in periods:
        returns_period = returns.loc[(returns.index >= period_start) & (returns.index <= period_end)]

        cumulative_returns_period = (1 + returns_period).cumprod() - 1
        key = str(period_name)
        cumulative_returns[key][strat_name] = cumulative_returns_period

period_name = periods[0][0]
key = str(period_name)

traces = []
for portfolio_name, cumulative_return in cumulative_returns[key].items():
    trace = go.Scatter(x=cumulative_return.index, y=cumulative_return, mode='lines', name=portfolio_name)
    traces.append(trace)

layout = go.Layout(
    title=f'{period_name} Cumulative Returns',
    xaxis=dict(title='Date', showline=True, showgrid=True, gridwidth=0.2, gridcolor='lightgray', showticklabels=True, linecolor='black'),
    yaxis=dict(title='Cumulative Return', showline=True, showgrid=True, gridwidth=0.2, gridcolor='lightgray', showticklabels=True, linecolor='black'),
    legend=dict(
        orientation="h",
        x=0.5,
        y=-0.15,
        xanchor='center',
        yanchor='top',
        bgcolor='rgba(0,0,0,0)'),
)

fig = go.Figure(data=traces, layout=layout)

fig.add_shape(
    type="rect",
    x0=0,
    x1=1,
    y0=0,
    y1=1,
    xref="paper",
    yref="paper",
    line=dict(color="black", width=1.5),
    layer="below"
)

fig.update_layout(
    plot_bgcolor='white',
    paper_bgcolor='white',
    legend_title='',
)

fig.show()


In [ ]:
strat_stats["Period 2"]

In [ ]:
results_period2

In [ ]:
period_name = periods[1][0]
key = str(period_name) 

traces = []
for portfolio_name, cumulative_return in cumulative_returns[key].items():
    trace = go.Scatter(x=cumulative_return.index, y=cumulative_return, mode='lines', name=portfolio_name)
    traces.append(trace)

layout = go.Layout(
    title=f'{period_name} Cumulative Returns',
    xaxis=dict(title='Date', showline=True, showgrid=True, gridwidth=0.2, gridcolor='lightgray', showticklabels=True, linecolor='black'),
    yaxis=dict(title='Cumulative Return', showline=True, showgrid=True, gridwidth=0.2, gridcolor='lightgray', showticklabels=True, linecolor='black'),
    legend=dict(
        orientation="h",
        x=0.5,
        y=-0.15,
        xanchor='center',
        yanchor='top',
        bgcolor='rgba(0,0,0,0)'),
)

fig = go.Figure(data=traces, layout=layout)

fig.add_shape(
    type="rect",
    x0=0,
    x1=1,
    y0=0,
    y1=1,
    xref="paper",
    yref="paper",
    line=dict(color="black", width=1.5),
    layer="below"
)

fig.update_layout(
    plot_bgcolor='white',
    paper_bgcolor='white',
    legend_title='',
)

fig.show()


## Group Section: Combined Stratgy 

In [ ]:
# Organizing the Individual Strategies 

grouped_df = new_long_short
grouped_df = grouped_df[grouped_df['datadate'] >= '1992-04-14'].copy()
grouped_df = grouped_df.rename(columns={'portfolio_returns': 'longShort-Credit'})
grouped_df = grouped_df.iloc[:, :2]

df_base_t = portfolio_returns_longShort.iloc[:, :2].copy()

monthly_df = df_base_t['LongShortPortfolio'].resample('M').apply(lambda x: (1 + x).prod() - 1)

monthly_df = pd.DataFrame({'longShort-Consumer': monthly_df})

monthly_df.reset_index(inplace=True)

cols = ['Date'] + [col for col in monthly_df if col != 'Date']
monthly_df = monthly_df[cols]

monthly_df = monthly_df[monthly_df['Date'] <= '2017-02-28'].copy()

monthly_df = monthly_df.rename(columns={'Date': 'datadate'})

monthly_df['datadate'] = pd.to_datetime(monthly_df['datadate'])

grouped_df['datadate'] = pd.to_datetime(grouped_df['datadate'])

grouped_df = grouped_df.merge(monthly_df, on='datadate', how='left')

grouped_df.set_index('datadate', inplace=True)

data3_graph = data3[data3['datadate'] >= '1992-04-14'].copy()
data3_graph['datadate'] = pd.to_datetime(data3_graph['datadate'])

data3_subset = data3_graph[['datadate', 'mktrf']]

grouped_df_performance = pd.merge(grouped_df, data3_subset, on='datadate', how='left')

grouped_df_performance.set_index('datadate', inplace=True)

port_names = grouped_df_performance.columns
port_returns = grouped_df_performance


# Performance Metrics Comparison Individual Startegies 

results_list = []

for column in port_returns.columns:
    returns = port_returns[column]
    
    annual_return = calculate_annual_return(returns)
    annual_volatility = calculate_annualized_volatility(returns)
    
    sharpe_ratio = annual_return/annual_volatility
    
    results_list.append({
        'Column': column,
        'Annual Return': "{:.2f}%".format(annual_return * 100),
        'Annual Volatility': "{:.2f}%".format(annual_volatility * 100),
        'Sharpe Ratio': sharpe_ratio
    })

results_df = pd.DataFrame(results_list)

results_df

In [ ]:
# Market Cumulative 
data3_graph = data3[data3['datadate'] >= '1992-04-14'].copy()
data3_graph['Market'] = (1 + data3_graph['mktrf']).cumprod() - 1
data3_graph['datadate'] = pd.to_datetime(data3_graph['datadate'])

# Individual Startegies Cumulative
grouped_df_cum = (1 + grouped_df).cumprod() - 1
grouped_df_cum.reset_index(inplace=True)


# Plotting
fig = px.line(grouped_df_cum, x='datadate', y=grouped_df_cum.columns[1:], title='Graph 5: Strategy Portfolio Comparison')


fig.add_trace(go.Scatter(x=data3_graph['datadate'], y=data3_graph['Market'],
                         mode='lines', name='Market', line=dict(dash='dot')))

fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Portfolio Cumulative Return',
    xaxis=dict(showline=True, showgrid=True, gridwidth=0.2, gridcolor='lightgray', showticklabels=True, linecolor='black'),
    yaxis=dict(showline=True, showgrid=True, gridwidth=0.2, gridcolor='lightgray', showticklabels=True, linecolor='black'),
    plot_bgcolor='white',
    paper_bgcolor='white',
    legend_title='',
    legend=dict(
        orientation="h",
        x=0.5,
        y=-0.15,
        xanchor='center',
        yanchor='top',
        bgcolor='rgba(0,0,0,0)',
        font=dict(size=14)  
    ),
    title=dict(
        x=0.5,  
        y=0.9, 
    ),
    margin=dict(
        b=100,  
    )
)

fig.add_shape(
    type="rect",
    x0=0,
    x1=1,
    y0=0,
    y1=1,
    xref="paper",
    yref="paper",
    line=dict(color="black", width=1.5),
    layer="below"
)

fig.update_layout(width=800, height=600)

fig.show()


In [ ]:
# Correlation between the two strategies

correlation_longShort_Credit_Consumer = grouped_df['longShort-Credit'].corr(grouped_df['longShort-Consumer'])
correlation_longShort_Credit_Consumer 

## 50-50 Portfolio

In [ ]:
# Equal Weights Portfolio 

grouped_df.reset_index('datadate', inplace=True)
ew_grouped_df= grouped_df[grouped_df['datadate'] >= '1992-04-14'].copy()
ew_grouped_df.set_index('datadate', inplace=True)
ew_grouped_df = ew_grouped_df/ 2

ew_multi_strat = pd.DataFrame()

ew_multi_strat ['Equal Weight Portfolio'] = ew_grouped_df['longShort-Credit'] + ew_grouped_df['longShort-Consumer']

ew_multi_strat.reset_index('datadate', inplace=True)
ew_multi_strat_IN = ew_multi_strat[ew_multi_strat['datadate'] <= '1997-04-14'].copy()
ew_multi_strat_OUT = ew_multi_strat[ew_multi_strat['datadate'] > '1997-04-14'].copy()

ew_multi_strat_IN.set_index('datadate', inplace=True)
ew_multi_strat_OUT.set_index('datadate', inplace=True)

# Cumulative Returns

ew_multi_strat_cumIN = (1 + ew_multi_strat_IN).cumprod() - 1

ew_multi_strat_cumIN.reset_index('datadate', inplace=True)

ew_multi_strat_cumOUT = (1 + ew_multi_strat_OUT).cumprod() - 1

ew_multi_strat_cumOUT.reset_index('datadate', inplace=True)

## Tangency Portfolio & Minimum Variance Portfolio

In [ ]:
# Tangency Portfolio & Minimum Variance 

grouped_df_5_years = grouped_df[grouped_df['datadate'] <= '1997-04-14'].copy()

grouped_df_5_years['datadate'] = pd.to_datetime(grouped_df_5_years['datadate'], format='%Y%m%d')

grouped_df_5_years.set_index('datadate', inplace=True)

data3_5_years= data3[data3['datadate'] >= '1992-04-14'].copy()
data3_5_years= data3[data3['datadate'] <= '1997-04-14'].copy()

def portfolio_return(weights):
    return np.dot(grouped_df_5_years.mean(), weights) *12

def portfolio_std(weights):
    return (np.dot(np.dot(grouped_df_5_years.cov(), weights), weights))**(1/2) * np.sqrt(12)

def weights_creator(df):
    rand = np.random.random(len(grouped_df_5_years.columns))
    rand /= rand.sum()
    return rand

returns = []
stds = []
w = []

for i in range(10000):
    weights = weights_creator(grouped_df_5_years)
    returns.append(portfolio_return(weights))
    stds.append(portfolio_std(weights))
    w.append(weights)


sharpe_ratios = np.array(returns) / np.array(stds)

tangency_portfolio_idx = np.argmax(sharpe_ratios)

rf= data3_5_years['rf'].mean()

cml_stds = np.linspace(0.03, max(stds), 100)

cml_returns = rf + ((returns[tangency_portfolio_idx] - rf) / stds[tangency_portfolio_idx]) * cml_stds

# Plotting Efficient Frontier
plt.scatter(stds, returns, label='Efficient Frontier', s=20)
plt.scatter(min(stds), returns[stds.index(min(stds))], c="yellow", s=30, label="Min Variance Portfolio")
plt.scatter(stds[tangency_portfolio_idx], returns[tangency_portfolio_idx], c="red", s=30, label="Tangency Portfolio")

# Plotting the Capital Market Line (CML) 
cml_line, = plt.plot(cml_stds, cml_returns, label='Capital Market Line', color='green', linestyle='--', alpha=0.5)

plt.title("Graph 6: Efficient Frontier with CML")
plt.xlabel("Portfolio Annual Standard Deviation ")
plt.ylabel("Portfolio Annual Return")
plt.legend()
plt.show()

In [ ]:
# Tangency an Minimum Variance Weights 
pd.set_option('display.float_format', '{:.3f}'.format)

weights_df = pd.DataFrame({
    'Tangency Portfolio': w[tangency_portfolio_idx],
    'Min Variance Portfolio': w[np.argmin(stds)]
}, index=grouped_df_5_years.columns)

weights_df


In [ ]:
# Organizing the In-Sample Data 

grouped_df_IN = grouped_df[grouped_df['datadate'] < '1997-04-14'].copy()
mvp_return_df_IN = pd.DataFrame(grouped_df_IN)
tangency_return_df_IN = pd.DataFrame(grouped_df_IN)

mvp_return_df_IN = pd.DataFrame({'datadate': grouped_df_IN['datadate']})
tangency_return_df_IN= pd.DataFrame({'datadate': grouped_df_IN['datadate']})

mvp_return_df_IN['Min Variance Portfolio'] = grouped_df_IN['longShort-Credit']*weights_df['Min Variance Portfolio'][0]+grouped_df_IN['longShort-Consumer']*weights_df['Min Variance Portfolio'][1]
tangency_return_df_IN['Tangency Portfolio'] = grouped_df_IN['longShort-Credit']*weights_df['Tangency Portfolio'][0]+grouped_df_IN['longShort-Consumer']*weights_df['Tangency Portfolio'][1]

mvp_return_df_IN.set_index('datadate', inplace=True)
tangency_return_df_IN.set_index('datadate', inplace=True)

mvp_return_cum_IN = (1 + mvp_return_df_IN).cumprod() - 1
tangency_return_cum_IN = (1 + tangency_return_df_IN).cumprod() - 1

In [ ]:
# Organizing the Out-of-Sample data 

grouped_df_forward = grouped_df[grouped_df['datadate'] > '1997-04-14'].copy()
mvp_return_df = pd.DataFrame(grouped_df_forward)
tangency_return_df = pd.DataFrame(grouped_df_forward)

mvp_return_df = pd.DataFrame({'datadate': grouped_df_forward['datadate']})
tangency_return_df= pd.DataFrame({'datadate': grouped_df_forward['datadate']})

mvp_return_df['Min Variance Portfolio'] = grouped_df_forward['longShort-Credit']*weights_df['Min Variance Portfolio'][0]+grouped_df_forward['longShort-Consumer']*weights_df['Min Variance Portfolio'][1]
tangency_return_df['Tangency Portfolio'] = grouped_df_forward['longShort-Credit']*weights_df['Tangency Portfolio'][0]+grouped_df_forward['longShort-Consumer']*weights_df['Tangency Portfolio'][1]

mvp_return_df.set_index('datadate', inplace=True)
tangency_return_df.set_index('datadate', inplace=True)

mvp_return_cum = (1 + mvp_return_df).cumprod() - 1
tangency_return_cum = (1 + tangency_return_df).cumprod() - 1

## Multi-strat Comparison

In [ ]:
# PERFORMANCE METRICS IN-SAMPLE

multistrat_df_IN = pd.merge(ew_multi_strat_IN, tangency_return_df_IN, on='datadate', how='inner')
multistrat_df_IN = pd.merge(multistrat_df_IN,mvp_return_df_IN , on='datadate', how='inner')

data3_IN = data3[(data3['datadate'] > '1992-04-14') & (data3['datadate'] < '1997-04-14')].copy()
data3_IN['datadate'] = pd.to_datetime(data3_IN['datadate'])

data3_IN_subset = data3_IN[['datadate', 'mktrf']]

multistrat_df_IN = pd.merge(multistrat_df_IN, data3_IN_subset, on='datadate', how='left')

multistrat_df_IN.set_index('datadate', inplace=True)

port_name_IN = multistrat_df_IN.columns
port_returns_IN = multistrat_df_IN

results_list_IN = []

for column in port_returns_IN.columns:
    returns = port_returns_IN[column]
    
    annual_return = calculate_annual_return(returns)
    annual_volatility = calculate_annualized_volatility(returns)
    
    sharpe_ratio = annual_return/annual_volatility
    
    results_list_IN.append({
        '': column,
        'Annual Return': "{:.2f}%".format(annual_return * 100),
        'Annual Volatility': "{:.2f}%".format(annual_volatility * 100),
        'Sharpe Ratio': sharpe_ratio
    })

results_df_IN = pd.DataFrame(results_list_IN)

results_df_IN

In [ ]:
# PERFROMANCE METRICS OUT-OF-SAMPLE

multistrat_df_OUT = pd.merge(ew_multi_strat_OUT, tangency_return_df, on='datadate', how='inner')
multistrat_df_OUT = pd.merge(multistrat_df_OUT,mvp_return_df , on='datadate', how='inner')

data3_OUT = data3[data3['datadate'] > '1997-04-14'].copy()
data3_OUT['datadate'] = pd.to_datetime(data3_OUT['datadate'])

data3_OUT_subset = data3_OUT[['datadate', 'mktrf']]

multistrat_df_OUT = pd.merge(multistrat_df_OUT, data3_OUT_subset, on='datadate', how='left')

multistrat_df_OUT.set_index('datadate', inplace=True)

port_names_OUT = multistrat_df_OUT.columns
port_returns_OUT = multistrat_df_OUT

results_list_OUT = []

for column in port_returns_OUT.columns:
    returns = port_returns_OUT[column]
    
    annual_return = calculate_annual_return(returns)
    annual_volatility = calculate_annualized_volatility(returns)
    
    sharpe_ratio = annual_return/annual_volatility
    
    results_list_OUT.append({
        ' ': column,
        'Annual Return': "{:.2f}%".format(annual_return * 100),
        'Annual Volatility': "{:.2f}%".format(annual_volatility * 100),
        'Sharpe Ratio': sharpe_ratio
    })

results_df_OUT = pd.DataFrame(results_list_OUT)

results_df_OUT

In [ ]:
# CUMULATIVE IN-SAMPLE

import plotly.graph_objects as go
data3_IN = data3[(data3['datadate'] > '1992-04-14') & (data3['datadate'] < '1997-04-14')].copy()

multistrat_cum_dfIN = ew_multi_strat_cumIN.merge(tangency_return_cum_IN, on='datadate', how='left')
multistrat_cum_dfIN = multistrat_cum_dfIN.merge(mvp_return_cum_IN, on='datadate', how='left')

data3_insample = data3_IN[['datadate','mktrf']].copy()
data3_insample['datadate'] = pd.to_datetime(data3_insample['datadate'])
data3_insample['cumulative_mktrf'] = (1 + data3_insample['mktrf']).cumprod() - 1

# Plotting
fig = go.Figure()

for column in multistrat_cum_dfIN.columns[1:]:
    fig.add_trace(go.Scatter(x=multistrat_cum_dfIN['datadate'], y=multistrat_cum_dfIN[column], mode='lines', name=column))

fig.add_trace(go.Scatter(x=data3_insample['datadate'], y=data3_insample['cumulative_mktrf'],
                         mode='lines', name='Market', line=dict(dash='dot')))

fig.update_layout(
    title='Graph 7: In-Sample Cumulative Returns',
    title_y = 0.87,
    title_x = 0.5,  
    title_yanchor ='bottom',
    title_font=dict(size=20),
    xaxis_title='Date',
    yaxis_title ='Portfolio Cumulative Return',
    xaxis=dict(showline =True, showgrid=True, gridwidth=0.2, gridcolor='lightgray', showticklabels=True, linecolor='black'),
    yaxis=dict(showline =True, showgrid=True, gridwidth=0.2, gridcolor='lightgray', showticklabels=True, linecolor='black',
               zeroline =True, zerolinewidth=0.2, zerolinecolor='lightgray'),  # Add zeroline configuration
    plot_bgcolor='white',
    paper_bgcolor='white',
    legend_title='',
    legend=dict(
        orientation="h",
        x=0.5,
        y=-0.15,
        xanchor='center',
        yanchor='top',
        bgcolor='rgba(0,0,0,0)',
        font=dict(size=13) 
    ),
    shapes=[
        dict(
            type="rect",
            x0=0,
            x1=1,
            y0=0,
            y1=1,
            xref="paper",
            yref="paper",
            line=dict(color="black", width=1.5),
            layer="below"
        )
    ],  
    )


fig.update_layout(width=800, height=600)

fig.show()



In [ ]:
# CUMULATIVE OUT-OF-SAMPLE

import plotly.graph_objects as go
data3_OUT = data3[data3['datadate'] > '1997-04-14'].copy()
# Assuming 'datadate' is the date column in your DataFrame
multistrat_cum_dfOUT = ew_multi_strat_cumOUT.merge(tangency_return_cum, on='datadate', how='left')
multistrat_cum_dfOUT = multistrat_cum_dfOUT.merge(mvp_return_cum, on='datadate', how='left')

data3_outsample = data3_OUT[['datadate','mktrf']].copy()
data3_outsample['datadate'] = pd.to_datetime(data3_outsample['datadate'])
data3_outsample['cumulative_mktrf'] = (1 + data3_outsample['mktrf']).cumprod() - 1

# Plotting
fig = go.Figure()

# Add the Multi-Strategy and Long-Short Portfolio traces
for column in multistrat_cum_dfOUT.columns[1:]:
    fig.add_trace(go.Scatter(x=multistrat_cum_dfOUT['datadate'], y=multistrat_cum_dfOUT[column], mode='lines', name=column))

# Add the Market cumulative in a dotted line
fig.add_trace(go.Scatter(x=data3_outsample['datadate'], y=data3_outsample['cumulative_mktrf'],
                         mode='lines', name='Market', line=dict(dash='dot')))

fig.update_layout(
    title='Graph 8: Out-of-Sample Cumulative Returns',
    title_y = 0.87,
    title_x = 0.5,  
    title_yanchor ='bottom',
    title_font=dict(size=20),
    xaxis_title='Date',
    yaxis_title='Portfolio Cumulative Return',
    xaxis=dict(showline=True, showgrid=True, gridwidth=0.2, gridcolor='lightgray', showticklabels=True, linecolor='black'),
    yaxis=dict(showline=True, showgrid=True, gridwidth=0.2, gridcolor='lightgray', showticklabels=True, linecolor='black',
               zeroline=True, zerolinewidth=0.2, zerolinecolor='lightgray'),  # Add zeroline configuration
    plot_bgcolor='white',
    paper_bgcolor='white',
    legend_title='',
    legend=dict(
        orientation="h",
        x=0.5,
        y=-0.15,
        xanchor='center',
        yanchor='top',
        bgcolor='rgba(0,0,0,0)',
        font=dict(size=13) 
    ),
)
fig.add_shape(
    type="rect",
    x0=0,
    x1=1,
    y0=0,
    y1=1,
    xref="paper",
    yref="paper",
    line=dict(color="black", width=1.5),
    layer="below"
)

fig.update_layout(width=800, height=600)

fig.show()



In [ ]:
# Calculating the drawdowns OUT-OF-SAMPLE

def calculate_drawdowns(wealth_index):
    previous_peaks = wealth_index.cummax()
    drawdowns = (wealth_index - previous_peaks) / previous_peaks
    return drawdowns

drawdowns_ew = calculate_drawdowns(multistrat_cum_dfOUT['Equal Weight Portfolio']+1)
drawdowns_tan = calculate_drawdowns(multistrat_cum_dfOUT['Tangency Portfolio']+1)
drawdowns_min = calculate_drawdowns(multistrat_cum_dfOUT['Min Variance Portfolio']+1)

# Plotiing

fig = go.Figure()

fig.add_trace(go.Scatter(x=multistrat_cum_dfOUT['datadate'], y=drawdowns_ew, mode='lines', name='Equal Weight Portfolio'))
fig.add_trace(go.Scatter(x=multistrat_cum_dfOUT['datadate'], y=drawdowns_tan, mode='lines', name='Tangency Portfolio'))
fig.add_trace(go.Scatter(x=multistrat_cum_dfOUT['datadate'], y=drawdowns_min, mode='lines', name='Min Variance Portfolio'))

fig.update_layout(
    title='Graph 11: Portfolios Drawdowns',
    title_y = 0.8,
    title_x = 0.1,  
    title_yanchor ='bottom',
    title_font=dict(size=18),
    xaxis_title='Date',
    yaxis_title='Drawdown',
    paper_bgcolor='white',  
    plot_bgcolor='white',   
    xaxis=dict(linecolor='black', linewidth=1, mirror=True, gridcolor='lightgray', gridwidth=0.5),  
    yaxis=dict(linecolor='black', linewidth=1, mirror=True, gridcolor='lightgray', gridwidth=0.5, zeroline=True, zerolinecolor='lightgray', zerolinewidth=0.5), 
    legend=dict(x=0.75, y=0.01, traceorder='normal', bgcolor='white', bordercolor='black', borderwidth=0.2, font=dict(size=14)), 
    height=350, 
    width=1000,   
)

fig.show()

In [ ]:
# IN-SAMPLE
data3_IN_sub = data3_IN[['datadate', 'smb', 'hml', 'rf']].copy()  
data3_IN_sub['datadate'] = pd.to_datetime(data3_IN_sub['datadate'])
data3_IN_sub.set_index('datadate', inplace=True)

multi_market_df_IN = multistrat_df_IN.merge(data3_IN_sub, on='datadate', how='left')

# OUT-OF-SAMPLE
data3_OUT_sub = data3_OUT[['datadate', 'smb', 'hml', 'rf']].copy()  
data3_OUT_sub['datadate'] = pd.to_datetime(data3_OUT_sub['datadate'])
data3_OUT_sub.set_index('datadate', inplace=True)

multi_market_df_OUT = multistrat_df_OUT.merge(data3_OUT_sub, on='datadate', how='left')


In [ ]:
# CAPM IN-SAMPLE

results_CAPM_IN = []

multi_market_df_IN.iloc[:, :3] = multi_market_df_IN.iloc[:, :3].apply(pd.to_numeric, errors='coerce')

for column in multi_market_df_IN.columns[:3]:
    
    y = multi_market_df_IN[column]  
    X = multi_market_df_IN['mktrf']
    X = sm.add_constant(X)

    model = sm.OLS(y, X).fit()

    alpha = model.params['const']
    beta = model.params['mktrf']
    t_statistic_alpha = model.tvalues['const']
    t_statistic_beta = model.tvalues['mktrf']

    excess_returns = y
    tracking_error = excess_returns.std()
    ir = alpha / tracking_error

    results_CAPM_IN.append({
        'Portfolio': f'{column} ',
        'Alpha (CAPM)': alpha,
        'T-Statistic (Alpha)': t_statistic_alpha,
        'Beta (CAPM)': beta,
        'T-Statistic (Beta)': t_statistic_beta,
        'Information Ratio (CAPM)': ir,
    })

results_CAPM_df_IN = pd.DataFrame(results_CAPM_IN)
results_CAPM_df_IN

In [ ]:
# CAPM OUT-OF-SAMPLE

results_CAPM_OUT = []

multi_market_df_OUT.iloc[:, 0:3] = multi_market_df_OUT.iloc[:, 0:3].apply(pd.to_numeric, errors='coerce')

for column in multi_market_df_OUT.columns[0:3]:
    
    y = multi_market_df_OUT[column]  
    X = multi_market_df_OUT['mktrf']
    X = sm.add_constant(X)

    model = sm.OLS(y, X).fit()

    alpha = model.params['const']
    beta = model.params['mktrf']
    t_statistic_alpha = model.tvalues['const']
    t_statistic_beta = model.tvalues['mktrf']

    excess_returns = y
    tracking_error = excess_returns.std()
    ir = alpha / tracking_error

    results_CAPM_OUT.append({
        'Portfolio': f'{column} ',
        'Alpha (CAPM)': alpha,
        'T-Statistic (Alpha)': t_statistic_alpha,
        'Beta (CAPM)': beta,
        'T-Statistic (Beta)': t_statistic_beta,
        'Information Ratio (CAPM)': ir,
    })

results_CAPM_df_OUT = pd.DataFrame(results_CAPM_OUT)
results_CAPM_df_OUT

In [ ]:
# FF3 IN-SAMPLE

results_ff3_IN = []

multi_market_df_IN.iloc[:,0:3] = multi_market_df_IN.iloc[:,0:3].apply(pd.to_numeric, errors='coerce')

for column in multi_market_df_IN.columns[0:3]:
    y = multi_market_df_IN[column]
    X = multi_market_df_IN[['mktrf', 'smb', 'hml']]
    X = sm.add_constant(X)

    model = sm.OLS(y, X).fit()

    alpha = model.params['const']
    t_statistic_alpha = model.tvalues['const']

    beta_mkt = model.params['mktrf']
    t_statistic_mkt = model.tvalues['mktrf']

    beta_smb = model.params['smb']
    t_statistic_smb = model.tvalues['smb']

    beta_hml = model.params['hml']
    t_statistic_hml = model.tvalues['hml']

    excess_returns = y
    tracking_error = excess_returns.std()
    ir = alpha / tracking_error

    results_ff3_IN.append({
        'Portfolio': f'{column}',
        'Alpha (FF3)': alpha,
        'T-Statistic Alpha': t_statistic_alpha,
        'Beta Mkt-Rf': beta_mkt,
        'T-Statistic Mkt-Rf': t_statistic_mkt,
        'Beta SMB': beta_smb,
        'T-Statistic SMB': t_statistic_smb,
        'Beta HML': beta_hml,
        'T-Statistic HML': t_statistic_hml,
        'Information Ratio (FF3)': ir,
    })

results_FF3_IN = pd.DataFrame(results_ff3_IN)
results_FF3_IN


In [ ]:
# FF3 OUT-OF-SAMPLE

results_ff3_OUT = []

multi_market_df_OUT.iloc[:, 0:3] = multi_market_df_OUT.iloc[:, 0:3].apply(pd.to_numeric, errors='coerce')

for column in multi_market_df_OUT.columns[0:3]:
   
    y = multi_market_df_OUT[column]
    X = multi_market_df_OUT[['mktrf', 'smb', 'hml']]
    X = sm.add_constant(X)

    model = sm.OLS(y, X).fit()

    alpha = model.params['const']
    t_statistic_alpha = model.tvalues['const']

    beta_mkt = model.params['mktrf']
    t_statistic_mkt = model.tvalues['mktrf']

    beta_smb = model.params['smb']
    t_statistic_smb = model.tvalues['smb']

    beta_hml = model.params['hml']
    t_statistic_hml = model.tvalues['hml']

    excess_returns = y
    tracking_error = excess_returns.std()
    ir = alpha / tracking_error

    results_ff3_OUT.append({
        'Portfolio': f'{column}',
        'Alpha (FF3)': alpha,
        'T-Statistic Alpha': t_statistic_alpha,
        'Beta Mkt-Rf': beta_mkt,
        'T-Statistic Mkt-Rf': t_statistic_mkt,
        'Beta SMB': beta_smb,
        'T-Statistic SMB': t_statistic_smb,
        'Beta HML': beta_hml,
        'T-Statistic HML': t_statistic_hml,
        'Information Ratio (FF3)': ir,
    })

results_FF3_OUT = pd.DataFrame(results_ff3_OUT)
results_FF3_OUT



In [ ]:
# Splitting the out-of-sample

multi_strat_returns = multistrat_df_OUT

multi_strat_returns.sort_index(inplace=True)
multi_strat_returns.reset_index(inplace=True)

split_date = '2007-12-31'

period1 = multi_strat_returns[multi_strat_returns['datadate'] < '2007-12-31'].copy()
period2 = multi_strat_returns[multi_strat_returns['datadate'] > '2007-12-31'].copy()
period1.set_index('datadate', inplace=True)
period2.set_index('datadate', inplace=True)

In [ ]:
def calculate_metrics_for_columns(df):
    metrics = pd.DataFrame(index=df.columns, columns=['Annual Return', 'Annual Volatility', 'Sharpe Ratio'])

    for column in df.columns:
        annual_return = calculate_annual_return(df[column])
        annual_volatility = calculate_annualized_volatility(df[column])
        sharpe_ratio = annual_return / annual_volatility

        metrics.at[column, 'Annual Return'] = "{:.2f}%".format(annual_return * 100)
        metrics.at[column, 'Annual Volatility'] = "{:.2f}%".format(annual_volatility * 100)
        metrics.at[column, 'Sharpe Ratio'] = sharpe_ratio

    return metrics


# Performance metrics for Period 1

metrics_period1 = calculate_metrics_for_columns(period1)
metrics_period1


In [ ]:
# Performance metrics for Period 2

metrics_period2 = calculate_metrics_for_columns(period2)
metrics_period2

In [ ]:
# Cumulative returns in period 1

period1.drop(columns=['mktrf'], inplace=True)
cumulative_returns_period1 = period1.apply(lambda x: (1 + x).cumprod() - 1)

# Market cumulative 

data3_p1 = data3_OUT[['datadate', 'mktrf']].copy()
data3_p1['datadate'] = pd.to_datetime(data3_p1['datadate'])
data3_p1 = data3_p1[data3_p1['datadate'] <= '2007-12-31']
data3_p1['cumulative_mktrf'] = (1 + data3_p1['mktrf']).cumprod() - 1


# Plotting

fig = go.Figure()

for column in cumulative_returns_period1.columns:
    fig.add_trace(go.Scatter(x=cumulative_returns_period1.index, y=cumulative_returns_period1[column], mode='lines', name=column))

fig.add_trace(go.Scatter(x=data3_p1['datadate'], y=data3_p1['cumulative_mktrf'],
                         mode='lines', name='mktrf', line=dict(dash='dot')))


fig.update_layout(
    title=' Graph 9: Cumulative Returns - Period 1',
    title_y = 0.87,
    title_x = 0.5,  
    title_yanchor ='bottom',
    title_font=dict(size=20),
    xaxis_title='Date',
    yaxis_title='Cumulative Return',
    legend=dict(
        orientation="h",
        x=0.5,
        y=-0.15,
        xanchor='center',
        yanchor='top',
        bgcolor='rgba(0,0,0,0)',
        font=dict(size=13) 
    ),
    xaxis=dict(showline=True, showgrid=True, gridwidth=0.2, gridcolor='lightgray', showticklabels=True, linecolor='black'),
    yaxis=dict(showline=True, showgrid=True, gridwidth=0.2, gridcolor='lightgray', showticklabels=True, linecolor='black',
               zeroline=True, zerolinewidth=0.2, zerolinecolor='lightgray'),
    plot_bgcolor='white',
    paper_bgcolor='white',
)

fig.add_shape(
    type="rect",
    x0=0,
    x1=1,
    y0=0,
    y1=1,
    xref="paper",
    yref="paper",
    line=dict(color="black", width=1.5),
    layer="below"
)

fig.update_layout(width=800, height=600)
fig.show()

In [ ]:
# Cumulative Returns in period 2
period2.drop(columns=['mktrf'], inplace=True)
cumulative_returns_period2 = period2.apply(lambda x: (1 + x).cumprod() - 1)

# Market Cumulative Return
data3_p2 = data3_OUT[['datadate','mktrf']].copy()
data3_p2['datadate'] = pd.to_datetime(data3_p2['datadate'])
data3_p2 = data3_p2[data3_p2['datadate'] > '2007-12-31']
data3_p2['cumulative_mktrf'] = (1 + data3_p2['mktrf']).cumprod() - 1

#Plotting

fig = go.Figure()

for column in cumulative_returns_period2.columns:
    fig.add_trace(go.Scatter(x=cumulative_returns_period2.index, y=cumulative_returns_period2[column], mode='lines', name=column))

fig.add_trace(go.Scatter(x=data3_p2['datadate'], y=data3_p2['cumulative_mktrf'],
                         mode='lines', name='mktrf', line=dict(dash='dot')))

fig.update_layout(
    title='Graph 10: Cumulative Returns - Period 2',
    title_y = 0.87,
    title_x = 0.5,  
    title_yanchor ='bottom',
    title_font=dict(size=20),
    xaxis_title='Date',
    yaxis_title='Cumulative Return',
    legend=dict(
        orientation="h",
        x=0.5,
        y=-0.15,
        xanchor='center',
        yanchor='top',
        bgcolor='rgba(0,0,0,0)',
        font=dict(size=13) 
    ),
    xaxis=dict(showline=True, showgrid=True, gridwidth=0.2, gridcolor='lightgray', showticklabels=True, linecolor='black'),
    yaxis=dict(showline=True, showgrid=True, gridwidth=0.2, gridcolor='lightgray', showticklabels=True, linecolor='black',
               zeroline=True, zerolinewidth=0.2, zerolinecolor='lightgray'),
    plot_bgcolor='white',
    paper_bgcolor='white',
)

fig.add_shape(
    type="rect",
    x0=0,
    x1=1,
    y0=0,
    y1=1,
    xref="paper",
    yref="paper",
    line=dict(color="black", width=1.5),
    layer="below"
)

fig.update_layout(width=800, height=600)
fig.show()

In [ ]:
# Organizing the data for the FF3

data3_Reg1 = data3_OUT.copy()
data3_Reg1['datadate'] = pd.to_datetime(data3_Reg1['datadate'])
data3_Reg1 = data3_Reg1[data3_Reg1['datadate'] <= '2007-12-31']

data3_Reg2 = data3_OUT.copy()
data3_Reg2['datadate'] = pd.to_datetime(data3_Reg2['datadate'])
data3_Reg2 = data3_Reg2[data3_Reg2['datadate'] > '2007-12-31']

period1.reset_index(inplace=True)
period2.reset_index(inplace=True)

period1 = period1.merge(data3_Reg1[['datadate', 'mktrf', 'rf', 'smb', 'hml']], on='datadate', how='left')
period2 = period2.merge(data3_Reg2[['datadate', 'mktrf', 'rf', 'smb', 'hml']], on='datadate', how='left')

In [ ]:
# FF3 - period 1 
results_ff3 = []

period1.iloc[:, 1:4] = period1.iloc[:, 1:4].apply(pd.to_numeric, errors='coerce')

for column in period1.columns[1:4]:
    y_period1 = period1[column]
    X_period1 = period1[['mktrf','smb', 'hml']]
    X_period1 = sm.add_constant(X_period1)

    model_period1 = sm.OLS(y_period1, X_period1).fit()

    alpha_period1 = model_period1.params['const']
    t_statistic_alpha_period1 = model_period1.tvalues['const']

    beta_mkt_period1 = model_period1.params['mktrf']
    t_statistic_mkt_period1 = model_period1.tvalues['mktrf']

    beta_smb_period1 = model_period1.params['smb']
    t_statistic_smb_period1 = model_period1.tvalues['smb']

    beta_hml_period1 = model_period1.params['hml']
    t_statistic_hml_period1 = model_period1.tvalues['hml']

    excess_returns_period1 = y_period1
    tracking_error_period1 = excess_returns_period1.std()
    ir_period1 = alpha_period1 / tracking_error_period1

    results_ff3.append({
        'Portfolio': f'{column} - Period 1',
        'Alpha (FF3)': alpha_period1,
        'T-Statistic Alpha': t_statistic_alpha_period1,
        'Beta Mkt-Rf': beta_mkt_period1,
        'T-Statistic Mkt-Rf': t_statistic_mkt_period1,
        'Beta SMB': beta_smb_period1,
        'T-Statistic SMB': t_statistic_smb_period1,
        'Beta HML': beta_hml_period1,
        'T-Statistic HML': t_statistic_hml_period1,
        'Information Ratio (FF3)': ir_period1,
    })

results_FF3_period1 = pd.DataFrame(results_ff3)
results_FF3_period1


In [ ]:
# FF3 - period 2 
results_ff3 = []

period2.iloc[:, 1:4] = period2.iloc[:, 1:4].apply(pd.to_numeric, errors='coerce')

for column in period2.columns[1:4]:
    y_period2 = period2[column]
    X_period2 = period2[['mktrf','smb', 'hml']]
    X_period2 = sm.add_constant(X_period2)

    model_period2 = sm.OLS(y_period2, X_period2).fit()

    alpha_period2 = model_period2.params['const']
    t_statistic_alpha_period2 = model_period2.tvalues['const']

    beta_mkt_period2 = model_period2.params['mktrf']
    t_statistic_mkt_period2 = model_period2.tvalues['mktrf']

    beta_smb_period2 = model_period2.params['smb']
    t_statistic_smb_period2 = model_period2.tvalues['smb']

    beta_hml_period2 = model_period2.params['hml']
    t_statistic_hml_period2 = model_period2.tvalues['hml']

    excess_returns_period2 = y_period2
    tracking_error_period2 = excess_returns_period2.std()
    ir_period2 = alpha_period2 / tracking_error_period2

    results_ff3.append({
        'Portfolio': f'{column} - Period 2',
        'Alpha (FF3)': alpha_period2,
        'T-Statistic Alpha': t_statistic_alpha_period2,
        'Beta Mkt-Rf': beta_mkt_period2,
        'T-Statistic Mkt-Rf': t_statistic_mkt_period2,
        'Beta SMB': beta_smb_period2,
        'T-Statistic SMB': t_statistic_smb_period2,
        'Beta HML': beta_hml_period2,
        'T-Statistic HML': t_statistic_hml_period2,
        'Information Ratio (FF3)': ir_period2,
    })

results_FF3_period2 = pd.DataFrame(results_ff3)
results_FF3_period2
